# *Импорт библиотек*

In [ ]:
%%capture
!pip install deskew

In [ ]:
from google.colab import files # Импортируем Модуль files для работы с файлами
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-white')
sns.set_context('notebook', rc={'figure.figsize': (10, 8)}, font_scale=1.5)

import numpy as np # Импортируем библиотеку numpy
import pandas as pd

import time
import random
import math
import re
import os # Импортируем библиотеку os для раоты с фаловой системой
import shutil
import json
import cv2

from copy import deepcopy
from IPython.display import clear_output

# from tensorflow.keras.applications import MobileNetV3Small
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import one_hot
# from tensorflow.keras.preprocessing import image # Импортируем модуль image для работы с изображениями

from sklearn.model_selection import train_test_split

from deskew import determine_skew

In [ ]:
from PIL import Image, ImageFile, ImageOps # импортируем модель Image для работы с изображениями
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from google.colab import drive # Подключаем гугл-диск
drive.mount('/content/drive')

In [ ]:
# Контекстный менеджер для измерения времени операций
# Операция обертывается менеджером с помощью оператора with
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        # print('Время обработки: {:.2f} с'.format(time.time() - self.t))
        seconds_total = time.time() - self.t
        minutes = int(seconds_total // 60)
        seconds = (seconds_total % 60) + (seconds_total - math.floor(seconds_total))
        print('Время обработки: {} мин {:.2f} c'.format(minutes, seconds))

with timex():
    print()

In [ ]:
# Указываем путь к базе данных
directory = '/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/Scans'
directory_new = '/content/ClearedReshapedBase'

# Анализ базы


In [ ]:
# Указываем путь к базе данных
directory = '/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/Scans'

### Анализ количества файлов и их типов

In [ ]:
base_len = len(os.listdir(directory))
print(f'Всего файлов: {base_len}')

# Выведем информацию о типах файлов в датасете
extension_dict = dict()

for i in os.listdir(directory):
    ext = re.findall(r'\.[^.]+$', i)[0].lower()
    if ext not in extension_dict:
        extension_dict[ext] = 1
        continue

    extension_dict[ext] += 1

print('Кол-во файлов по типам расширения:')
for key, val in extension_dict.items():
    print(f'{key:5} - {val:5} ({val / base_len:.2%})')

# Выведем информацию о размере базы данных
size_list = [os.path.getsize(os.path.join(directory, file)) for file in os.listdir(directory)]
print(f"Общий размер файлов: {sum(size_list) / (1024 * 1024):.2f} MB")
print(f"Средний размер файла: {sum(size_list) / (1024 * 1024) / base_len:.2f} MB")

In [ ]:
def create_image_df(folder_path):

    # Функция для проверки на обрезанность фотографии
    def is_cropped_image(image_path):
        # Загрузка изображения
        image_ = cv2.imread(image_path)

        # Преобразование в оттенки серого и пороговая обработка
        gray = cv2.cvtColor(image_, cv2.COLOR_BGR2GRAY)
        _, thresholded = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

        # Морфологические операции для устранения шума и связанных компонент
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        thresholded = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)

        # Поиск контуров серой области
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Установка минимальных размеров для обрезанных областей
        min_width = 450
        min_height = 50

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w < min_width or h < min_height:
                # Если не найдена обрезанная область, вернуть 1
                return 1

        # Если найдены только обрезанные области, вернуть 0
        return 0

    files = os.listdir(folder_path)

    file_names = []
    file_size_kb = []
    file_size_mb = []
    image_width = []
    image_height = []
    orientation = []
    cropped = []
    shapes = []
    dpis = []

    for file in files:
        file_path = os.path.join(folder_path, file)
        # try:
        with Image.open(file_path) as img:
            file_names.append(file)
            file_size_kb.append(os.path.getsize(file_path) / 1024)
            file_size_mb.append(os.path.getsize(file_path) / (1024 * 1024))
            width, height = img.size
            image_width.append(width)
            image_height.append(height)
            if width > height:  orientation.append(0)  # Горизонтальное изображение
            else:               orientation.append(1)  # Вертикальное изображение

            try:
                if image.mode in ("RGB", "RGBA"):   shapes.append('Color')
                elif image.mode == "L":             shapes.append('BW')
                else:                               shapes.append('Unknown')
            except:
                shapes.append('Unknown')

            dpi = image.info.get("dpi", (0, 0))

            if dpi:
                dpis.append(dpi)
            else:
                dpis.append(0)

            cropped.append(is_cropped_image(file_path))

        # except Exception as e:
        #     print(f"Ошибка при обработке изображения {file}: {e}")

    data = {
        'Название файла': file_names,
        'Размер файла (KB)': file_size_kb,
        'Размер файла (MB)': file_size_mb,
        'Ширина изображения': image_width,
        'Высота изображения': image_height,
        'dpi':dpi,
        'Ориентация': orientation,
        'Цветность':shapes,
        'Обрезан':cropped,
    }

    return pd.DataFrame(data)

with timex():
    df = create_image_df(directory)

In [ ]:
# Выбираем дубликаты, исключая столбец 'Название файла'
duplicates = df[df.duplicated(subset=df.columns.difference(['Название файла']), keep=False)]

duplicates

# **Преобразование базы к единому размеру**
> Нахождение оптимального DPI \
> Создание новой базы, где изображения будут в оттенках серого и одного размера \
> Разделение данных на батчи (с учётом сортировки для удобной разметки) \

## **Анализ изображений в базе**

### Создание датафрейма с информацией о базе

In [ ]:
def DPI2px(dpi):
    print('DPI {:3}: {:4} px, {:4} px'.format(dpi, round(dpi * (210 / 25.4)), round(dpi * (297 / 25.4))))

print('         Ширина    Высота')
for i in [50,100,150,200,250,300,350,600]:
    DPI2px(i)

* Смотрим, сколько изображений уменьшим по высоте, определяем, уменьшаем ли изображения нормального разрешения.
* Убираем белый фон с права и с лева по максимуму, оставляем немного белых пикселей по ширине с двух сторон.
* По высоте, если снизу много белых пикселей, то оставляем до нужной нам высоты А4.
* Проверяем работу deskew на выделенной группе изображений.
* Повернуть изображения в группе rotated, на нужный угол (instance_rotate).
* Находим лучший метод интерполяции для увеличения и уменьшения изображений.

In [ ]:
# Создаем пустой датафрейм
df1 = pd.DataFrame(columns=["File Name", "New File Name", "Ширина (px)", "Высота (px)", "DPI_X", "DPI_Y", "DPI_X_A4", "DPI_Y_A4", 'aspect_ratio', 'rotate_angle'])

# Продвинутый перевод изображения в массив
def to_numpy(im):
    im.load()
    # unpack data
    e = Image._getencoder(im.mode, 'raw', im.mode)
    e.setimage(im.im)

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(im)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))

    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)
        mem[offset:offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data

# Функция для получения информации о изображении
def get_image_info(file_path, angle=False):
    img = Image.open(file_path)
    width, height = img.size
    dpi = img.info.get("dpi", (0, 0))
    aspect_ratio = width / height
    if angle == True:
        angle = determine_skew(cv2.cvtColor(to_numpy(img), cv2.COLOR_BGR2GRAY))
        img.close()

        return width, height, dpi[0], dpi[1], aspect_ratio, angle

    img.close()

    return width, height, dpi[0], dpi[1], aspect_ratio

# height, width, channels = img.shape

# Разрешение A4 в миллиметрах
a4_width_mm = 210
a4_height_mm = 297

files_len = len(os.listdir(directory))

# Перебираем файлы в папке
for index, filename in enumerate(os.listdir(directory)):
    file_path = os.path.join(directory, filename)
    width, height, dpi_x, dpi_y, aspect_ratio = get_image_info(file_path)
    dpi_x_a4 = width / (a4_width_mm / 25.4)  # Переводим мм в дюймы
    dpi_y_a4 = height / (a4_height_mm / 25.4)  # Переводим мм в дюймы
    # df1 = pd.concat([df1, pd.DataFrame({"Имя файла": [filename], "Ширина (px)": [width], "Высота (px)": [height], "DPI_X": [dpi_x], "DPI_Y": [dpi_y], "DPI_X_A4": [dpi_x_a4], "DPI_Y_A4": [dpi_y_a4], 'aspect_ratio': [aspect_ratio], 'rotate':indices[index]})], ignore_index=True)
    df1 = pd.concat([df1, pd.DataFrame({"File Name": [filename], "Ширина (px)": [width], "Высота (px)": [height], "DPI_X": [dpi_x], "DPI_Y": [dpi_y], "DPI_X_A4": [dpi_x_a4], "DPI_Y_A4": [dpi_y_a4], 'aspect_ratio': [aspect_ratio]})], ignore_index=True) # 'rotate_angle':[rotate_angle]

    if index % 200 == 0:
        clear_output()
        print(f'Готово {index} из {files_len}')

df1['DPI_X'] = df1['DPI_X'].astype(int)
df1['DPI_Y'] = df1['DPI_Y'].astype(int)
df1['Ширина (px)'] = df1['Ширина (px)'].astype(int)
df1['Высота (px)'] = df1['Высота (px)'].astype(int)

##### **Находим углы поворота (deskew)**

In [ ]:
def compute_skew1(img1): # определние не большого поворота отностильно горизонта с помощью готового решения deskew
    grayscale_image = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    angle = determine_skew(grayscale_image)
    print(angle)
    # определили угол и повернем на него изображение
    img_rot = img1.copy()
    height, width = img_rot.shape[:2]
    # get the center coordinates of the image to create the 2D rotation matrix
    center = (width/2, height/2)
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=angle, scale=1)
    img_rot = cv2.warpAffine(src=img_rot, M=rotate_matrix, dsize=(width, height))

    return img_rot

In [ ]:
def compute_skew_angle(img1): # определние не большого поворота отностильно горизонта с помощью готового решения deskew
    grayscale_image = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    angle = determine_skew(grayscale_image)

    return angle

def compute_skew_angle_1(img1):
    grayscale_image = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    angle = determine_skew(grayscale_image, **{'min_deviation': 0.01, 'num_peaks': 100, 'max_angle':7.5, 'min_angle':-7.5,})

    return angle

In [ ]:
angles_2 = []
time_per_image_2 = []

for i in df1.index[:5]:
    original_image = to_numpy(Image.open(directory + '/' + df1.iloc[i]['File Name']))
    #original_image = cv2.imread(directory + '/' + df1.iloc[i]['File Name'])
    try:
        t = time.time()
        angles_2.append(compute_skew_angle(original_image))
        time_per_image_2.append(time.time()-t)
    except:
        angles_2.append(0)

In [ ]:
angles_2 = []
time_per_image_2 = []

for i in df1.index[:4413]:
    original_image = to_numpy(Image.open(directory + '/' + df1.iloc[i]['File Name']))
    #original_image = cv2.imread(directory + '/' + df1.iloc[i]['File Name'])
    try:
        t = time.time()
        angles_2.append(compute_skew_angle(original_image))
        time_per_image_2.append(time.time()-t)
    except:
        angles_2.append(0)

    if i % 100 == 0:
        clear_output()
        print(i)

# ~ 60-70 мс на открытие изображения на прогретой машине = 7 минут на все 5980 изображений
# ~ 3000-5000 мс на нахождение угла поворота изображения = ~5-8.3 ЧАСА на 5980 изображений

In [ ]:
angles_1 = []
time_per_image = []

for i in df1.index[4413:]:
    original_image = to_numpy(Image.open(directory + '/' + df1.iloc[i]['File Name']))
    # original_image = cv2.imread(directory + '/' + df1.iloc[i]['File Name'])

    try:
        t = time.time()
        angles_1.append(compute_skew_angle(original_image))
        time_per_image.append(time.time()-t)
    except:
        angles_1.append(0)

    if i % 100 == 0:
        clear_output()
        print(i)

In [ ]:
angles = angles_2 + angles_1

In [ ]:
time_per_image_ = time_per_image_2 + time_per_image

In [ ]:
df1['deskew_angle'] = angles
df1['deskew_time_to_find_angle'] = time_per_image_

##### Пробуем модель определения поворота

In [ ]:
from keras.models import load_model

# Загрузка модели
model = load_model('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/my_model.h5')

def load_img_cv2(img_path, single=False):
    # Загрузка изображения и изменение размера, для модели поворота изображений
    image = cv2.imread(img_path)
    if len(image.shape) != 3 or image.shape[2] != 3:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image = cv2.resize(image, (224, 224))
    image_array = np.array(image) / 255.0

    if single==True:
        image_array = np.expand_dims(image_array, axis=0)

    return image_array

def prepare_imgs(img_paths):
    return np.array([load_img_cv2(img_path) for img_path in img_paths])

In [ ]:
# Делаем предикты
files_dirs = [directory + '/' + i for i in os.listdir(directory)]
index = 0
predicts = []
while index < len(files_dirs):
    predicts += list(model.predict(prepare_imgs(files_dirs[index:index+100])))

    if index % 100 == 0:
        clear_output()
        print(f'Готово {index} из {len(files_dirs)}')

    index += 100

indices = np.argmax(predicts, axis=1)

##### ***Добавление дополнительных столбцов***

In [ ]:
large_text_angle = [7,9,25,46,76,79,91,92,94,95,99,108,110,129,139,142,144,147,
	150,170,172,175,178,179,180,183,203,210,212,214,222,229,234,
	236,241,249,251,259,260,267,269,275,276,284,285,297,300,313,
	314,321,323,329,334,343,349,357,358,376,379,383,386,387,388,
	390,392,409,418,422,440,441,445,455,468,471,473,480,482,485,
	487,492,495,502,513,516,517,533,540,544,560,565,582,583,613,
	620,621,625,626,628,638,639,662,663,684,694,696,702,707,711,
	718,721,732,733,739,740,741,742,749,753,754,756,758,770,787,
	796,799,803,805,808,815,825,826,829,836,844,854,869,883,884,
	888,889,898,901,902,903,904,905,908,912,913,920,925,926,931,
	938,941,946,949,979,980,984,994,995,997,1035,1052,1053,1099,
	1103,1111,1120,1123,1130,1139,1168,1170,1188,1192,1210,1214,
	1236,1238,1239,1247,1252,1259,1260,1262,1267,1271,1288,1290,
	1292,1300,1302,1303,1309,1312,1328,1333,1335,1339,1347,1348,
	1349,1350,1352,1358,1362,1370,1378,1386,1393,1401,1404,1417,
	1420,1421,1422,1424,1431,1440,1448,1450,1451,1477,1478,1485,
	1490,1505,1523,1537,1538,1545,1548,1549,1550,1553,1569,1570,
	1578,1579,1585,1586,1589,1590,1598,1599,1605,1606,1607,1609,
	1615,1623,1622,1636,1638,1669,1679,1683,1687,1689,1693,1694,
	1718,1719,1732,1758,1759,1790,1791,1793,1794,1812,1825,1854,
	1870,1889,1890,1895,1899,1912,1919,1937,1946,1944,1950,1961,
	1965,1966,1979,1988,2012,2026,2029,2030,2035,2040,2055,2060,
	2066,2085,2089,2090,2093,2098,2101,2104,2103,2107,2133,2138,
	2151,2167,2173,2211,2209,2256,2309,2313,2326,2345,2350,2372,
	2379,2381,2391,2397,2406,2408,2429,2447,2450,2454,2468,2471,
	2480,2584,2590,2606,2611,2612,2614,2626,2649,2700,2703,2708,
	2725,2743,2744,2745,2759,2787,2809,2812,2898,2903,2904,2928,
	2942,2951,2952,2957,2958,2960,2976,3005,3006,3007,3015,3016,
	3054,3062,3064,3068,3115,3124,3131,3142,3140,3137,3234,3251,
	3261,3286,3299,3314,3318,3319,3329,3331,3332,3335,3344,3350,
	3353,3354,3357,3358,3366,3371,3372,3374,3398,3399,3404,3405,
	3406,3415,3416,3417,3418,3420,3426,3430,3432,3436,3437,3438,
	3444,3443,3449,3450,3451,3461,3479,3492,3493,3494,3497,3498,
	3500,3506,3507,3508,3515,3519,3533,3535,3537,3540,3545,3546,
	3547,3549,3550,3551,3552,3556,3556,3559,3562,3563,3570,3572,
	3575,3577,3578,3579,3583,3584,3588,3589,3591,3600,3608,3614,
	3615,3618,3620,3621,3622,3625,3626,3628,3629,3631,3632,
	3633,3634,3635,3635,3639,3643,3644,3648,3652,3653,3662,
	3671,3673,3674,3678,3679,3686,3687,3688,3689,3704,3507,
	3707,3708,3709,3719,3729,3737,3743,3751,3769,3772,3774,
	3780,3781,3782,3790,3792,3793,3796,3798,3799,3802,3806,
	3812,3817,3818,3819,3826,3831,3833,3866,3868,3938,3986,
	3987,3988,3989,3992,4026,4029,4030,4031,4033,4034,4035,
	4036,4052,4059,4071,4074,4078,4085,4086,4087,4107,4152,
	4155,4156,4212,4210,4227,4247,4248,4249,4250,4342,4345,
	4349,4391,4469,4491,4540,4547,4549,4566,4573,4574,4607,
	4635,4636,4637,4654,4667,4830,4910,5099,5116,5240,5251,
	5275,5317,5326,5365,5431,5483,5484,5613,5688,5689,5691,
	5698,5699,5700,5701,5705,5712,5715,5730,5731,5733,5734,
	5755,5778,5779,5801,5802,5803,5809,5813,5814,5816,5868,
	5899,5900,5904,5977,
	]

rotated = [128,170,183,184,524,526,538,1207,1237,1239,2153,3019,3325,3371,
	3375,3698,4178,5128,5366,5424,5444,5461,5867,]

defect = [12,17,20,34,35,38,67,77,87,88,97,99,119,125,137,
	150,153,160,180,183,192,199,225,229,252,254,291,321,403,430,438,445,466,
	474,509,518,521,541,546,548,569,575,584,598,599,623,635,653,656,673,674,
	675,699,704,707,739,754,769,789,863,875,879,884,899,910,924,925,926,939,
	940,963,970,972,1010,1013,1014,1054,1088,
	1114,1140,1155,1208,1211,1225,1258,1270,1273,1285,1291,1301,1305,1306,
	1309,1315,1319,1321,1326,1327,1329,1331,1330,1338,1342,1343,1347,1356,
	1368,1374,1386,1388,1419,1426,1431,1433,1435,1456,1460,1462,1465,1468,
	1475,1477,1480,1481,1486,1492,1499,1500,1506,1511,1513,1518,1533,1534,
	1535,1537,1542,1548,1552,1556,1558,1559,1565,1566,1571,1572,1582,1583,
	1588,1589,1594,1595,1596,1598,1600,1609,1610,1618,1624,1623,1628,1630,
	1643,1644,1647,1648,1650,1651,1652,1667,1673,1676,1678,1680,1681,1683,
	1684,1686,1687,1691,1706,1710,1711,1713,1714,1733,1735,1739,1750,1753,
	1754,1755,1760,1761,1763,1765,1767,1089,1856,1860,1863,1865,1867,1869,
	1877,1878,1882,1884,1885,1887,1889,1890,1895,1897,1904,1911,1913,1915,
	1931,1946,1949,1953,1959,2000,2003,2004,2015,2016,2024,2022,2025,2039,
	2038,2052,2054,2056,2057,2067,2069,2077,2079,2080,2083,2084,2085,2094,
	2097,2106,2108,2109,2110,2115,2123,2132,2182,2189,2200,2201,2202,2211,
	2214,2227,2233,2262,2279,2289,2299,2300,2304,2328,2341,2352,2353,2356,
	2361,2372,2391,2397,2402,2404,2405,2407,2419,2434,2439,2440,2445,2447,
	2455,2483,2484,2485,2489,2491,2517,2560,2567,2574,2590,2591,2597,2602,
	2623,2626,2650,2680,2695,2700,2710,2713,2740,2744,2751,2752,2758,2796,
	2824,2857,2909,2917,2925,2926,2931,2939,2981,3089,3109,3153,3171,3184,
	3186,3189,3196,3224,3269,3298,3314,3322,3323,3334,3342,3345,3347,3349,
	3351,3352,3356,3357,3366,3369,3380,3381,3382,3383,3384,3385,3388,3389,
	3390,3392,3394,3401,3409,3414,3432,3436,3437,3438,3443,3462,3463,3465,
	3470,3471,3473,3474,3476,3497,3503,3509,3510,3517,3521,3524,3527,3532,
	3533,3540,3541,3544,3545,3546,3549,3555,3565,3566,3567,3577,3578,3579,
	3581,3583,3586,3589,3592,3595,3600,3610,3611,3612,3613,3614,3615,3616,
	3624,3629,3626,3631,3637,3638,3651,3654,3669,3673,3682,3683,3685,3701,
	3702,3709,3718,3719,3720,3721,3722,3723,3724,3725,3740,3741,3753,3756,
	3757,3758,3764,3769,3777,3778,3779,3784,3810,3812,3813,3814,3821,3829,
	3836,3840,3839,3848,3857,3858,3896,3905,3927,3928,3929,3930,3932,3935,
	3947,3958,3960,3966,3980,3983,3984,3985,3986,3987,3989,3994,4002,4003,
	4004,4025,4027,4028,4029,4036,4037,4042,4046,4066,4069,4078,4080,4086,
	4090,4125,4149,4159,4160,4165,4176,4177,4186,4187,4198,4217,4218,4219,
	4222,4233,4234,4236,4244,4249,4252,4259,4260,4261,4264,4265,4267,4268,
	4275,4285,4286,4287,4291,4292,4293,4300,4301,4302,4304,4305,4310,4314,
	4316,4348,4353,4356,4358,4364,4366,4382,4384,4402,4405,4409,4410,4411,
	4412,4415,4418,4419,4420,4421,4425,4430,4433,4437,4451,4457,4458,4459,
	4476,4482,4497,4502,4511,4515,4545,4549,4555,4556,4558,4562,4564,4567,
	4575,4576,4580,4581,4583,4584,4586,4593,4600,4606,4608,4609,4611,4616,
	4623,4624,4625,4626,4629,4630,4631,4643,4650,4652,4654,4662,4667,4673,
	4674,4681,4689,4700,4752,4823,4885,4886,4898,5055,5090,5093,5097,5100,
	5105,5110,5113,5114,5115,5116,5135,5139,5141,5155,5156,5223,5236,5237,
	5238,5248,5251,5254,5276,5295,5299,5316,5317,5329,5332,5358,5380,5391,
	5399,5421,5429,5430,5452,5457,5458,5470,5471,5473,5475,5476,5480,5483,
	5484,5490,5499,5502,5508,5531,5578,5581,5582,5585,5606,5607,5622,5623,
	5624,5625,5626,5627,5630,5631,5636,5668,5672,5693,5694,5696,5700,5702,
	5710,5714,5731,5734,5735,5738,5752,5753,5756,5768,5771,5780,5784,5785,
	5787,5789,5790,5798,5799,5806,5816,5817,5823,5826,5830,5851,5880,
	5885,5899,5900,5906,5908,5921,5938,]

cropped = [11,146,156,247,335,361,442,477,636,778,892,959,982,1000,
	1034,1039,1041,1047,1045,1050,1077,1084,1091,1113,1174,1240,1245,1351,
	1384,1427,1437,1489,1494,1535,1568,1573,1643,1644,1682,1975,2050,2146,2205,
	2229,2230,2265,2285,2286,2296,2308,2329,2366,2378,2388,2448,2451,2458,2465,
	2472,2475,2482,2507,2620,2763,2791,2813,2837,2843,2869,2901,2935,2940,2947,
	2971,3006,3025,3146,3156,3239,3288,3293,3315,3317,3330,3325,3355,3360,3495,
	3525,3586,3595,3603,3623,3630,3677,3738,3766,3822,3824,3842,3848,3855,3867,
	3909,3957,3983,4027,4034,4051,4085,4103,4139,4191,4210,4211,4213,4232,4276,
	4277,4282,4315,4319,4330,4331,4333,4335,4355,4371,4377,4384,4429,4467,4508,
	4517,4548,4605,4612,4634,4639,4694,4733,4789,4934,4945,4952,4961,4985,4989,
	4994,5250,5370,5382,5394,5411,5459,5532,5633,5644,5657,5670,5714,5782,5783,
	5875,5878,5901,5929,5945,]

In [ ]:
def get_null_ones_list(l):
    return [1 if i in l else 0 for i in range(len(df1))]

large_text_angle = get_null_ones_list(large_text_angle)
rotated = get_null_ones_list(rotated)
defect = get_null_ones_list(defect)
cropped = get_null_ones_list(cropped)

In [ ]:
df1['large_rotate_angle'] = large_text_angle
df1['need_rotate'] = rotated
df1['defect'] = defect
df1['cropped'] = cropped

##### ***Находим угол для изображений, которые нужно перевернуть***
> **Отодельно для PIL и cv2 из-за разного положения, при открытии изображения.**\
> *PIL открывает некоторые изображения в отличающейся от оригинальной ориентации.*



In [ ]:
df1[df1['need_rotate'] == 1]

In [ ]:
create_rotated_image_grid_cv2(0, directory + '/' + 'invoice_84_22.02.2019_170240000298_061241008162.jpg', True)

In [ ]:
def create_rotated_image_grid_cv2(index, image_path, use_PIL=False):
    # Загрузка оригинального изображения с помощью OpenCV
    if use_PIL == False:
        original_image = cv2.imread(image_path)
    elif use_PIL == True:
        original_image = np.array(Image.open(image_path))

    # Повороты изображения
    rotated_90 = cv2.rotate(original_image, cv2.ROTATE_90_CLOCKWISE)
    rotated_180 = cv2.rotate(original_image, cv2.ROTATE_180)
    rotated_minus_90 = cv2.rotate(original_image, cv2.ROTATE_90_COUNTERCLOCKWISE)

    # Создаем график с 1x4 подграфиками
    fig, axes = plt.subplots(1, 4, figsize=(36, 16), facecolor='lightgrey')

    # Подписи для каждого подграфика
    labels = ['orig' + [' cv2', ' PIL'][use_PIL] + f' ({index})', '90°', '180°', '-90°']

    # Отображаем оригинальное изображение и его повороты
    for i, (image, label) in enumerate(zip([original_image, rotated_90, rotated_180, rotated_minus_90], labels)):
        axes[i].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # OpenCV использует BGR, а Matplotlib ожидает RGB
        axes[i].set_title(label, fontsize=36)
        axes[i].axis('off')

    # Сохраняем график или выводим его на экран
    plt.tight_layout()
    plt.show()

for i in df1[df1['need_rotate'] == 1].index:
    print(i)
    # Замените 'your_image.jpg' на путь к вашему изображению
    create_rotated_image_grid_cv2(i,directory + '/' + df1.iloc[i]['File Name'])
    create_rotated_image_grid_cv2(i,directory + '/' + df1.iloc[i]['File Name'],True)

In [ ]:
instant_rotation_cv2 = {5867:-90,5366:90,4178:180,3698:180,3375:90,3325:-90,3019:180,2153:180,1239:90,1237:-90,1207:90,183:-90,170:-90,128:-90}
instant_rotation_PIL = {5867:-90,5366:90,5128:-90,4178:180,3698:180,3375:90,3371:180,3325:-90,3019:180,2153:180,1239:90,1237:90,1207:90,538:180,526:180,524:180,184:180,183:-90,170:-90,128:-90}

instant_rotation_cv2 = [instant_rotation_cv2[i] if i in instant_rotation_cv2 else 0 for i in range(len(df1))]
instant_rotation_PIL = [instant_rotation_PIL[i] if i in instant_rotation_PIL else 0 for i in range(len(df1))]

In [ ]:
df1['instant_rotation_cv2'] = instant_rotation_cv2
df1['instant_rotation_PIL'] = instant_rotation_PIL

##### *Примеры данных и статистика по датафрейму*

In [ ]:
df1.head()

In [ ]:
df1.describe()

In [ ]:
print('{:50}{:3} ({:7.3%})'.format('Обрезанные изображения: ', len(df1[df1['cropped'] == 1]), len(df1[df1['cropped'] == 1]) / len(df1)))
print('{:50}{:3} ({:7.3%})'.format('Дефектные изображения: ', len(df1[df1['defect'] == 1]), len(df1[df1['defect'] == 1]) / len(df1)))
print('{:50}{:3} ({:7.3%})'.format('Изображения которые необходимо повернуть: ', len(df1[df1['need_rotate'] == 1]), len(df1[df1['need_rotate'] == 1]) / len(df1)))
print('{:50}{:3} ({:7.3%})'.format('Изображения с большим углом поворота: ', len(df1[df1['large_rotate_angle'] == 1]), len(df1[df1['large_rotate_angle'] == 1])  / len(df1)))

In [ ]:
df1.quantile(0.1)

In [ ]:
df1[df1['DPI_X'] != df1['DPI_Y']]

In [ ]:
df1[df1['DPI_Y_A4'] >= 299]

##### *Анализ количества изображений, которые мы уменьшаем*


In [ ]:
target_hight = 3508
target_width = 2480

In [ ]:
print('Кол-во изображений для уменьшения по высоте:', len(df1[df1['Высота (px)'] > target_hight]))
print('Кол-во изображений для уменьшения по ширине:', len(df1[df1['Ширина (px)'] > target_width]))

##### **Анализ разных методов увеличения и уменьшения изображений**
> Лучшие для: \
> Уменьшения изображений - cv2.INTER_AREA \
> Увеличения изображений - cv2.INTER_CUBIC, cv2.INTER_LINEAR, cv2.INTER_LANCZOS4

In [ ]:
np.random.seed(42)

In [ ]:
# Список путей к файлам
filelist = [directory + '/' + i for i in os.listdir(directory)]

In [ ]:
img_name = np.random.choice(df1[df1['DPI_Y_A4'] < 200]['File Name'])
im = cv2.imread(directory + '/' + img_name)
# im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
print(im.shape)

In [ ]:
interpolation_algorithm = [
    ("nearest", cv2.INTER_NEAREST),
    ("bilinear", cv2.INTER_LINEAR),
    ("bicubic", cv2.INTER_CUBIC),
    ("area", cv2.INTER_AREA),
    ("lanczos4", cv2.INTER_LANCZOS4)
]

In [ ]:
def resize_test(img, factor, is_plot=True, file_name=None):
    if len(img.shape) == 2:
        height, width= img.shape
    elif len(img.shape) == 3:
        height, width, channels = img.shape
    height2, width2 = int(height*factor), int(width*factor)
    print('orig size:', height, width)
    print('resize size:', height2, width2)
    imgs = []
    for alg in interpolation_algorithm:
        img_r = cv2.resize(img, (width2, height2), interpolation = alg[1])
        imgs.append(img_r)
    if is_plot:
        plt.figure(figsize=(88,28))
        plt.subplot(1, len(imgs)+1, 1)
        plt.title('orig' + f' ({height}x{width})', fontsize=40)
        plt.imshow(img)
        plt.axis('off')
        for i in range( len(imgs) ):
            plt.subplot(1, len(imgs)+1, i+2)
            plt.title( interpolation_algorithm[i][0] + f' ({height2}x{width2})', fontsize=40)
            plt.imshow(imgs[i])
            plt.axis('off')
        # plt.subplots_adjust(wspace=0)
        if file_name != None:
            plt.savefig(file_name)
        plt.show()
    return imgs

print('DOWN')
img_down0 = resize_test(im, 0.75, file_name='opencv_resize_down0.png')
print('DOWN')
img_down1 = resize_test(im, 0.5, file_name='opencv_resize_down1.png')
print('DOWN DOWN')
img_down2 = resize_test(im, 0.25, file_name='opencv_resize_down2.png')
print('UP')
img_up0 = resize_test(img_down1[0], 4, file_name='opencv_resize_up1.png')
print('UP ORIG')
img_up0 = resize_test(im, 4, file_name='opencv_resize_up2.png')

##### Тест deskew

In [ ]:
import math
from typing import Tuple, Union

import cv2
import numpy as np

from deskew import determine_skew


def rotate(
        image: np.ndarray, angle: float, background: Union[int, Tuple[int, int, int]]
) -> np.ndarray:
    old_width, old_height = image.shape[:2]
    angle_radian = math.radians(angle)
    width = abs(np.sin(angle_radian) * old_height) + abs(np.cos(angle_radian) * old_width)
    height = abs(np.sin(angle_radian) * old_width) + abs(np.cos(angle_radian) * old_height)

    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    rot_mat[1, 2] += (width - old_width) / 2
    rot_mat[0, 2] += (height - old_height) / 2

    return cv2.warpAffine(image, rot_mat, (int(round(height)), int(round(width))), borderValue=background)

image = cv2.imread('input.png')
grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
angle = determine_skew(grayscale)
rotated = rotate(image, angle, (0, 0, 0))
cv2.imwrite('output.png', rotated)

In [ ]:
def compute_skew(img1, custom_settings=None): # определние не большого поворота отностильно горизонта с помощью готового решения deskew
    grayscale_image = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    if custom_settings == None:
        angle = determine_skew(grayscale_image)
    else:
        angle = determine_skew(grayscale_image, **custom_settings)

    # определили угол и повернем на него изображение
    img_rot = img1.copy()
    height, width = img_rot.shape[:2]
    # get the center coordinates of the image to create the 2D rotation matrix
    center = (width/2, height/2)
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=angle, scale=1)
    img_rot = cv2.warpAffine(src=img_rot, M=rotate_matrix, dsize=(width, height))

    return img_rot, angle

In [ ]:
import inspect

def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

In [ ]:
get_default_args(determine_skew)

In [ ]:
{'sigma': 3.0,
 'num_peaks': 20,
 'num_angles': None,
 'angle_pm_90': False,
 'min_angle': None,
 'max_angle': None,
 'min_deviation': 1.0}

In [ ]:
def create_rotated_image_grid_cv2(index, image_path, use_PIL=False, deskew_settings=None, save_index=None):
    # Загрузка оригинального изображения с помощью OpenCV
    if use_PIL == False:
        original_image = cv2.imread(image_path)
    elif use_PIL == True:
        original_image = np.array(Image.open(image_path))

    def get_string_from_dict(d):
        string = '('
        for key, val in d.items():
            string += str(key) + '=' + str(val) + ', '

        return string[:-2] + ')'

    # Поворачиваем изображение
    img_rot, angle = compute_skew(original_image)

    # Находим повороты изображения с другими настройками
    additional_imgs_list = []
    additional_imgs_angle_list = []
    if deskew_settings != None:
        for setting in deskew_settings:
            img_rot_1, angle_1 = compute_skew(original_image, setting)
            additional_imgs_list.append(img_rot_1)
            additional_imgs_angle_list.append(angle_1)

    # Создаем график с 1x2 подграфиками
    fig, axes = plt.subplots(1, 2 + len(additional_imgs_list), figsize=(18 + (9 * len(additional_imgs_list)), 16 + (8 * len(additional_imgs_list))), facecolor='lightgrey')

    # Подписи для каждого подграфика
    labels = ['orig' + [' cv2', ' PIL'][use_PIL] + f' ({index})', 'Rotated ' + f'{angle:.8f}'] + [get_string_from_dict(deskew_settings[i])[:20] + '...' + '\n' + 'Rotated ' + f'{angle1:.8f}' for i, angle1 in enumerate(additional_imgs_angle_list)]

    # Отображаем оригинальное изображение и его повороты
    for i, (image, label) in enumerate(zip([original_image, img_rot] + additional_imgs_list, labels)):
        axes[i].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # OpenCV использует BGR, а Matplotlib ожидает RGB
        axes[i].set_title(label, fontsize=8 + (8 * len(additional_imgs_list)))
        axes[i].axis('off')

    # Сохраняем график или выводим его на экран
    plt.tight_layout()
    if save_index != None:
        plt.savefig(f"/content/drive/MyDrive/imgs_2/{save_index}.png")
    else:
        plt.show()

    plt.clf()

In [ ]:
deskew_settings = [
    {'min_deviation': 0.5, 'num_peaks': 100, 'max_angle':7.5, 'min_angle':-7.5,},
    {'min_deviation': 0.1, 'num_peaks': 100, 'max_angle':7.5, 'min_angle':-7.5,},
    {'min_deviation': 0.1, 'num_peaks': 100, 'max_angle':7.5, 'min_angle':-7.5,'sigma': 2.0},
    {'min_deviation': 0.05, 'num_peaks': 100, 'max_angle':7.5, 'min_angle':-7.5,},
    {'min_deviation': 0.01, 'num_peaks': 100, 'max_angle':7.5, 'min_angle':-7.5,},
]

for index, i in enumerate(df1[df1['large_rotate_angle'] == 1].index):
    try:
        create_rotated_image_grid_cv2(i,directory + '/' + df1.iloc[i]['File Name'], deskew_settings=deskew_settings, save_index=index)
    except:
        break

##### Находим кол-во белых пикселей на изображении

In [ ]:
def get_white_pixels_percent(original_image):
    count = np.count_nonzero(np.all(original_image == [255, 255, 255], axis=2))
    return count, count / (original_image.shape[0] * original_image.shape[1])

In [ ]:
white_pixels = []
white_pixels_per = []
img_shapes = []

for i in df1.index:
    original_image = to_numpy(Image.open(directory + '/' + df1.iloc[i]['File Name']).convert('RGB'))
    img_shapes.append(len(original_image.shape))

    try:
        wp, wpp = get_white_pixels_percent(original_image)
    except:
        white_pixels.append(0)
        white_pixels_per.append(0)

    white_pixels.append(wp)
    white_pixels_per.append(wpp)

    if i % 100 == 0:
        clear_output()
        print(i)

In [ ]:
print(len(white_pixels))
print(len(white_pixels_per))
print(len(img_shapes))

In [ ]:
df1['channels'] = img_shapes
df1['white_pixel_count'] = white_pixels
df1['white_pixel_%'] = white_pixels_per

### *Сохранение/загрузка df1*

In [ ]:
# Сохранение
df1.to_csv('/content/drive/MyDrive/df_info.csv', index=False)

In [ ]:
# Загрузка
df1 = pd.read_csv('/content/drive/MyDrive/df1.csv')

In [ ]:
df1.head()

In [ ]:
df1.sort_values(by='white_pixel_%', ascending=False)

In [ ]:
df1.describe()

In [ ]:
err_list = ['0206.png','1806.png','3306.png','3967.png','4401.png','4751.png','5089.png','5717.png']

In [ ]:
df1[df1['filename_new'].isin(err_list)]

### Графики

In [ ]:
def plot_distribution(df, column_name):
    plt.figure(figsize=(10, 6))
    sns.histplot(df[column_name], kde=True)
    plt.title(f'Distribution of {column_name}')
    plt.xlabel(column_name)
    plt.ylabel('Count')
    plt.show()

def plot_quantile_distribution(df, column_name):
    plt.figure(figsize=(10, 6))
    sns.boxplot(y=df[column_name])
    plt.title(f'Quantile Distribution of {column_name}')
    plt.ylabel(column_name)
    plt.show()

In [ ]:
for column in df1.columns[1:]:
    try:    plot_distribution(df1, column)
    except: pass
    try:    plot_quantile_distribution(df1, column)
    except: pass

In [ ]:
def plot_comparison_box_plot(df):
    plt.figure(figsize=(10, 6))
    df[['DPI_X', 'DPI_Y', 'DPI_X_A4', 'DPI_Y_A4']].boxplot()
    plt.ylabel('Значение DPI')
    plt.title('Сравнение DPI_X, DPI_Y, DPI_X_A4 и DPI_Y_A4')
    plt.xticks(rotation=90)
    plt.show()

plot_comparison_box_plot(df1)

In [ ]:
def plot_quantile_values(df):
    plt.figure(figsize=(16, 8))
    plt.style.use('seaborn-darkgrid')

    columns = ['DPI_X', 'DPI_X_A4', 'DPI_Y', 'DPI_Y_A4']
    # quantiles = np.arange(0, 1.02, 0.02)  # Значения квантилей
    quantiles = np.arange(0, 1.1, 0.1)  # Значения квантилей
    q = deepcopy(quantiles)
    quantiles *= 100
    q = [round(j,2) for k, j in enumerate(q) if k in [_ for _, i in enumerate(quantiles) if round(i) <= 100]]
    quantiles = [round(i) for i in quantiles if round(i) <= 100]

    quantile_values = np.percentile(df[columns], quantiles, axis=0)

    for i, col in enumerate(columns):
        plt.plot(q, quantile_values[:, i], marker='o', label=col, alpha=0.35, linewidth=5, markeredgewidth=6)

    plt.xlabel('Квантиль', fontsize=18)
    plt.ylabel('Значение квантиля', fontsize=18)
    plt.title('Значения квантилей для столбцов', fontsize=24)
    plt.xticks(q, fontsize=14, rotation=90)
    plt.yticks(fontsize=16)
    plt.legend()
    plt.grid(True)
    plt.show()

# Пример использования
plot_quantile_values(df1)

In [ ]:
def plot_quantile_values(df):
    plt.figure(figsize=(16, 8))
    plt.style.use('seaborn-darkgrid')

    columns = ['DPI_X', 'DPI_X_A4', 'DPI_Y', 'DPI_Y_A4']
    # quantiles = np.arange(0, 1.02, 0.02)  # Значения квантилей
    quantiles = np.arange(0, 0.38, 0.01)  # Значения квантилей
    q = deepcopy(quantiles)
    quantiles *= 100
    q = [round(j,2) for k, j in enumerate(q) if k in [_ for _, i in enumerate(quantiles) if round(i) <= 100]]
    quantiles = [round(i) for i in quantiles if round(i) <= 100]

    quantile_values = np.percentile(df[columns], quantiles, axis=0)

    for i, col in enumerate(columns):
        plt.plot(q, quantile_values[:, i], marker='o', label=col, alpha=0.35, linewidth=5, markeredgewidth=6)

    plt.xlabel('Квантиль', fontsize=18)
    plt.ylabel('Значение квантиля', fontsize=18)
    plt.title('Значения квантилей для столбцов', fontsize=24)
    plt.xticks(q, fontsize=14, rotation=90)
    plt.yticks(fontsize=16)
    plt.legend()
    plt.grid(True)
    plt.show()

# Пример использования
plot_quantile_values(df1)

In [ ]:
def plot_quantile_values(df):
    plt.figure(figsize=(16, 8))
    plt.style.use('seaborn-darkgrid')

    columns = ['aspect_ratio']
    quantiles = np.arange(0, 1.02, 0.02)  # Значения квантилей
    # quantiles = np.arange(0, 0.38, 0.01)  # Значения квантилей
    q = deepcopy(quantiles)
    quantiles *= 100
    q = [round(j,2) for k, j in enumerate(q) if k in [_ for _, i in enumerate(quantiles) if round(i) <= 100]]
    quantiles = [round(i) for i in quantiles if round(i) <= 100]

    quantile_values = np.percentile(df[columns], quantiles, axis=0)

    for i, col in enumerate(columns):
        plt.plot(q, quantile_values[:, i], marker='o', label=col, alpha=0.35, linewidth=5, markeredgewidth=6)
        plt.plot(q, [0.707 for i in range(len(quantile_values[:, i]))], label='aspect_ratio_A4', alpha=0.35, linewidth=5, markeredgewidth=6)

    plt.xlabel('Квантиль', fontsize=18)
    plt.ylabel('Значение квантиля', fontsize=18)
    plt.title('Значения квантилей для столбцов', fontsize=24)
    plt.xticks(q, fontsize=14, rotation=90)
    plt.yticks(fontsize=16)
    plt.legend()
    plt.grid(True)
    plt.show()

# Пример использования
plot_quantile_values(df1)

In [ ]:
def plot_dual_distribution(column1, column2, label1, label2, color1, color2):
    plt.figure(figsize=(20, 12))
    plt.style.use('seaborn-darkgrid')

    # График для column1
    sns.histplot(column1, kde=True, color=color1, label=label1, alpha=0.6, binwidth=8, element='step', fill=True,)

    # График для column2
    sns.histplot(column2, kde=True, color=color2, label=label2, alpha=0.6, binwidth=8, element='step', fill=True,)

    plt.xlabel('Значение', fontsize=22)
    plt.ylabel('Плотность', fontsize=22)
    plt.title(f'Распределение {label1} и {label2}', fontsize=26)
    plt.legend(fontsize=22)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.xlim(0,450)
    plt.show()

# Пример использования для 'DPI_X' и 'DPI_X_A4'
plot_dual_distribution(df1['DPI_X'], df1['DPI_X_A4'], 'DPI_X', 'DPI_X_A4', 'skyblue', 'lightcoral')

# Пример использования для 'DPI_Y' и 'DPI_Y_A4'
plot_dual_distribution(df1['DPI_Y'], df1['DPI_Y_A4'], 'DPI_Y', 'DPI_Y_A4', 'lightgreen', 'gold')

In [ ]:
def count_and_plot_column_values(df, column_name, x_label_size=12, y_label_size=12, x_tick_size=10, y_tick_size=10):
    # Подсчитываем количество значений в столбце
    value_counts = df[column_name].value_counts()

    # Создаем столбчатую диаграмму с помощью Seaborn
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=column_name)

    # Настраиваем размер текста для подписей и тиков
    plt.title(f'Распределение значений в столбце "{column_name}"', fontsize=x_label_size)
    plt.xlabel('Значение', fontsize=x_label_size)
    plt.ylabel('Количество', fontsize=y_label_size)

    plt.xticks(fontsize=x_tick_size)
    plt.yticks(fontsize=y_tick_size)

    plt.show()

def calculate_and_plot_mse(df, true_values, predicted_values, x_label_size=12, y_label_size=12, x_tick_size=10, y_tick_size=10):
    # Вычисляем MSE
    mse = mean_squared_error(true_values, predicted_values)

    # Создаем столбчатую диаграмму для отображения MSE
    plt.figure(figsize=(6, 4))
    plt.bar(['MSE'], [mse])

    # Настраиваем размер текста для подписей и тиков
    plt.title('Mean Squared Error (MSE)', fontsize=x_label_size)
    plt.ylabel('Value', fontsize=y_label_size)

    plt.xticks(fontsize=x_tick_size)
    plt.yticks(fontsize=y_tick_size)

    plt.show()

In [ ]:
df1.drop(['DPI_X_Difference'], axis=1, inplace=True)

In [ ]:
# numeric_columns = df1.select_dtypes(include=['number'])
numeric_columns = df1[df1.columns[1:]]
description = df1.describe()

# Вывод 0.9 и 0.1 перцентилей для числовых столбцов
percentiles_09 = df1.quantile(0.9)
percentiles_01 = df1.quantile(0.1)

print("Описательная статистика:")
print(description)

print("\n0.9 перцентили:")
print(percentiles_09)

print("\n0.1 перцентили:")
print(percentiles_01)

### Примеры изображений разных DPI

In [ ]:
min_v = 250
max_v = 300
df1[df1['DPI_Y_A4'] > min_v][df1['DPI_Y_A4'] < max_v]

In [ ]:
def show_sample_images(dataframe, column_name, min_value, max_value, num_images, image_size=(8, 4)):
    # Шаг 1: Выборка из DataFrame по указанному столбцу и его минимальному и максимальному значению
    filtered_df = dataframe[(dataframe[column_name] >= min_value) & (dataframe[column_name] <= max_value)]

    # Шаг 2: Выборка указанного числа случайных значений из DataFrame
    selected_rows = random.sample(range(len(filtered_df)), min(num_images, len(filtered_df)))
    selected_data = filtered_df.iloc[selected_rows]

    # Шаг 3: Открываем изображения и отображаем их в одном ряду
    images = []
    data_for_title = []
    col_index = list(dataframe.columns).index(column_name)
    for _, row in selected_data.iterrows():
        img_path = row['Имя файла']
        data_for_title.append(row[col_index])
        image = Image.open(directory + '/' + img_path)
        images.append(image)

    # Создаем общий график с выбранными изображениями
    fig, ax = plt.subplots(1, len(images), figsize=image_size, facecolor='lightgrey')
    for i, image in enumerate(images):
        ax[i].imshow(image)
        ax[i].set_title('{}: {:.2f} ({}-{})'.format(column_name, data_for_title[i], min_value, max_value), fontsize=64)
        ax[i].axis('off')

    plt.show()

In [ ]:
df1[df1['aspect_ratio'] > 0.757]

In [ ]:
show_sample_images(df1, 'aspect_ratio', 0.757, 1, 5, (128,64))

In [ ]:
show_sample_images(df1, 'aspect_ratio', 1, 2, 5, (128,64))

In [ ]:
show_sample_images(df1, 'aspect_ratio', 1, 2, 5, (128,64))

In [ ]:
show_sample_images(df1, 'aspect_ratio', 1, 2, 5, (128,64))

In [ ]:
show_sample_images(df1, 'aspect_ratio', 0, 0.5, 5, (128,64))

In [ ]:
# DPI 300 имеют больше всего документов
show_sample_images(df1, 'DPI_Y_A4', 300, 350, 5, (128,64))

In [ ]:
show_sample_images(df1, 'DPI_Y_A4', 250, 300, 5, (128,64))

In [ ]:
show_sample_images(df1, 'DPI_Y_A4', 200, 250, 5, (128,64))

In [ ]:
show_sample_images(df1, 'DPI_Y_A4', 100, 150, 5, (128,64))

In [ ]:
show_sample_images(df1, 'DPI_Y_A4', 0, 100, 5, (128,64))

In [ ]:
show_sample_images(df1, 'DPI_Y_A4', 0, 75, 5, (128,64))

In [ ]:
show_sample_images(df1, 'DPI_Y_A4', 0, 70, 5, (128,64))

In [ ]:
for x in ['равен', 'больше', 'меньше']:
    for y in ['равен', 'больше', 'меньше']:
        print(f'{x=:6}, {y=:6}')

In [ ]:
# x=равен , y=равен         - нечего
# x=равен , y=больше        - убираем лишние белые пиксели по y, уменьшаем по высоте белые пиксели
# x=равен , y=меньше        - увеличиваем по y пропорционально, уменьшаем по ширине белые пиксели
# x=больше, y=равен         - убираем уменьшаем по ширине белые пиксели
# x=больше, y=больше        - уменьшаем по y пропорционально, уменьшаем по ширине белые пиксели (если меньше 300 dpi)
# x=больше, y=меньше        - увеличиваем по y пропорционально, уменьшаем по ширине белые пиксели (если меньше 300 dpi)
# x=меньше, y=равен         - уменьшаем по высоте белые пиксели
# x=меньше, y=больше        -
# x=меньше, y=меньше        -

Разные повороты изображений

In [ ]:
show_sample_images(df1[df1['rotate'] != 0], 'rotate', 1, 6, 10, (128,64))

### Выводы

* Начиная с 66% изображений имеют DPI >= 300 (3965 из 5980)
* Если смотреть изображения по DPI ниже 300, то:
* * На изображениях 200-300 DPI встречаются изображения плохого качества.
* * Ниже 200 DPI документы с маленьким текстом могут быть не читабельные.
* * Ниже 150 DPI большенство не читается, если маленький текст (напрмер в таблицах).

In [ ]:
min_v = 200
max_v = 250
df1[df1['DPI_Y_A4'] > min_v][df1['DPI_Y_A4'] < max_v]

In [ ]:
df1.index

In [ ]:
df1[df1['DPI_Y_A4'] > min_v][df1['DPI_Y_A4'] < max_v].describe()

In [ ]:
df1.iloc[0]['Имя файла']

In [ ]:
df1.iloc[0]

In [ ]:
def plot_images_with_labels(image_path1, image_path2, figsize=(10, 5)):
    # Открываем изображения с помощью Pillow
    img1 = Image.open(image_path1)
    img2 = Image.open(image_path2)

    # Создаем новый график с указанным размером
    fig, axes = plt.subplots(1, 2, figsize=figsize)

    # Добавляем обводку синего цвета по краям изображений
    img1_with_border = ImageOps.expand(img1, border=10, fill='blue')
    img2_with_border = ImageOps.expand(img2, border=10, fill='blue')

    # Отображаем изображения с обводкой
    axes[0].imshow(img1_with_border)
    axes[0].set_title(f"Оригинал: {img1.size[0]}x{img1.size[1]}", fontsize=48)
    axes[0].axis('off')

    axes[1].imshow(img2_with_border)
    axes[1].set_title(f"Преобразованное: {img2.size[0]}x{img2.size[1]}", fontsize=48)
    axes[1].axis('off')

    # Отображаем график
    plt.show()

# Пример использования
image_path1 = directory + '/' + input_path
image_path2 = output_path

plot_images_with_labels(image_path1, image_path2, figsize=(64, 48))

In [ ]:
# Пример использования
image_path1 = directory + '/' + input_path
image_path2 = output_path

plot_images_with_labels(image_path1, image_path2, figsize=(64, 48))

## **Создание итоговой базы, разделение на батчи**

In [ ]:
# Продвинутый перевод изображения в массив
def to_numpy(im):
    im.load()
    # unpack data
    e = Image._getencoder(im.mode, 'raw', im.mode)
    e.setimage(im.im)

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(im)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))

    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)
        mem[offset:offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data

In [ ]:
# Указываем путь к базе данных
directory = '/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/Scans'

In [ ]:
# Путь к корневой папке, где будут создаваться подпапки
root_path = '/content/drive/MyDrive/ClearedReshapedBase/'

# Создаем папки в диапазоне от 0 до 6000 с шагом 100
for i in range(0, 6000, 100):
    folder_name = f'{i}-{i+100}'
    folder_path = os.path.join(root_path, folder_name)
    os.makedirs(folder_path, exist_ok=True)

print("Папки успешно созданы.")

In [ ]:
df1.sort_values(by=['Высота (px)', 'Ширина (px)', 'aspect_ratio'], ascending=True)['File Name']

In [ ]:
file_indexes = [i+100 for i in range(0,6000,100)]
print(file_indexes)

In [ ]:
# определили угол и повернем на него изображение
img_rot = img1.copy()
height, width = img_rot.shape[:2]
# get the center coordinates of the image to create the 2D rotation matrix
center = (width/2, height/2)
rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=angle, scale=1)
img_rot = cv2.warpAffine(src=img_rot, M=rotate_matrix, dsize=(width, height))

In [ ]:
df1.sort_values(by=['white_pixel_%','height','width'], ascending=False).index[128]

In [ ]:
list(df1[df1['instant_rotation_PIL'].isin([-90,90])]['filename'])

In [ ]:
df1[df1['instant_rotation_PIL'] != 0]

In [ ]:
def to_numpy(im):
    im.load()
    # unpack data
    e = Image._getencoder(im.mode, 'raw', im.mode)
    e.setimage(im.im)

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(im)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))

    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)
        mem[offset:offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data

def process_one_image(img_path, save_path, filename, new_height, deskew_angle, instant_angle):
    # Открываем изображение
    img = Image.open(img_path).convert('RGBA')

    # Проверка на успешное чтение изображения
    if img is None:
        raise Exception(f"Ошибка чтения изображения: ({i})")

    # Находим ширину и высоту
    width, height = img.size

    # Переводим в массив (чтобы использовать cv2)
    img = to_numpy(img)

    # Конвертируем в .png
    _, img_png = cv2.imencode('.png', img)
    img = cv2.imdecode(img_png, cv2.IMREAD_UNCHANGED)

    # Поворачиваем изображение на найденный угол
    if instant_angle in [90,-90]:
        height, width = width, height
        if instant_angle == 90:
            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        elif instant_angle == -90:
            img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        rotate_matrix = cv2.getRotationMatrix2D(center=(width/2, height/2), angle=deskew_angle, scale=1)
    else:
        rotate_matrix = cv2.getRotationMatrix2D(center=(width/2, height/2), angle=deskew_angle+instant_angle, scale=1)

    img = cv2.warpAffine(src=img, M=rotate_matrix, dsize=(width, height))

    # Флаг того, уменьшаем ли мы изображение
    if height > new_height:
        downscale = 1
    else:
        downscale = 0

    # Определяем новую ширину изображений и соотношение сторон
    aspect_ratio = width / height
    width = int(new_height * aspect_ratio)

    # Изменение размера с сохранением пропорций
    if downscale == 1:
        img = cv2.resize(img, (width, new_height), interpolation=cv2.INTER_AREA)
    elif downscale == 0:
        img = cv2.resize(img, (width, new_height), interpolation=cv2.INTER_CUBIC)

    # Преобразовываем изображение в чёрно-белое
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Cохранение изменённого изображения
    cv2.imwrite(save_path + filename, img)

In [ ]:
img_path = [directory + '/' + i for i in list(df1[df1['instant_rotation_PIL'].isin([-90,90])]['filename'])]
save_path = '/content/'
filename = list(df1[df1['instant_rotation_PIL'].isin([-90,90])]['filename_new'])
deskew_angle = list(df1[df1['instant_rotation_PIL'].isin([-90,90])]['deskew_angle'])
instant_angle = list(df1[df1['instant_rotation_PIL'].isin([-90,90])]['instant_rotation_PIL'])
new_height = 3508           # Новая высота

for i in range(len(img_path)):
    process_one_image(img_path[i], save_path, filename[i], new_height, deskew_angle[i], instant_angle[i])

In [ ]:
root_path = '/content/drive/MyDrive/ClearedReshapedBase/'
file_folders = os.listdir(root_path)                        # Папки
file_indexes = [i+100 for i in range(0,6000,100)]           # Макс. значения для каждой папки
folder_i = 0                # Индекс папки для создания названия
names_list = []             # Список для новых названий файлов
error_index_list = []       # Список для индексов изображений с ошибкой

new_height = 3508           # Новая высота

sorted_indexes = df1.sort_values(by=['white_pixel_%','height','width'], ascending=False).index

with timex():
    for i, filename in enumerate(list(df1.sort_values(by=['white_pixel_%','height','width'], ascending=False)['filename'])):
        # Увеличиваем индекс папки в которую будем сохранять изображение
        while i > file_indexes[folder_i] - 1:
            folder_i += 1

        # Открываем изображение
        img = to_numpy(Image.open(directory + '/' + filename).convert('RGBA'))
        # img = cv2.imread(directory + '/' + filename, cv2.IMREAD_UNCHANGED)

        # Конвертируем в .png
        _, img_png = cv2.imencode('.png', img)
        img = cv2.imdecode(img_png, cv2.IMREAD_UNCHANGED)

        # Проверка на успешное чтение изображения
        if img is None:
            error_index_list.append(i)
            print(f"Ошибка чтения изображения: ({i})")
            continue

        # Находим ширину, высоту и соотношение сторон
        height, width, aspect_ratio = df1.iloc[sorted_indexes[i]]['height'], df1.iloc[sorted_indexes[i]]['width'], df1.iloc[sorted_indexes[i]]['aspect_ratio']

        # Поворачиваем изображение на найденный угол
        center = (width/2, height/2)
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle= df1.iloc[sorted_indexes[i]]['instant_rotation_PIL'] + df1.iloc[sorted_indexes[i]]['deskew_angle'], scale=1)
        img = cv2.warpAffine(src=img, M=rotate_matrix, dsize=(width, height))

        # Флаг того, уменьшаем ли мы изображение
        if height > new_height:
            downscale = 1
        else:
            downscale = 0

        # Определяем новую ширину изображений
        width = int(new_height * aspect_ratio)

        # Изменение размера с сохранением пропорций
        if downscale == 1:
            img = cv2.resize(img, (width, new_height), interpolation=cv2.INTER_AREA)
        elif downscale == 0:
            img = cv2.resize(img, (width, new_height), interpolation=cv2.INTER_CUBIC)

        # Создаём название для файла (полный путь)
        output_path = root_path + file_folders[folder_i] + '/' + f'{i:0>4}' + '.png'

        # Преобразовываем изображение в чёрно-белое
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # print(img.shape)

        # Cохранение изменённого изображения
        cv2.imwrite(output_path, img)

        if i % 100 == 0:
            clear_output()
            print(f'{i}/5980')

In [ ]:
df1.sort_values(by=['white_pixel_%','height','width'], ascending=False)

In [ ]:
names_list[:3]

In [ ]:
sorted_indexes[:3]

In [ ]:
n_dict = {i: j for i, j in zip(sorted_indexes, names_list)}
n_dict

In [ ]:
ni = [n_dict[i] for i in range(5980)]
print(ni[:5])

In [ ]:
sorted_indexes = df1.sort_values(by=['white_pixel_%','height','width'], ascending=False).index

In [ ]:
names_list = [f'{i:0>4}.png' for i in range(5980)]

In [ ]:
ni = [names_list[i] for i in sorted_indexes]

In [ ]:
df1['filename_new'] = ni

In [ ]:
n_dict = {i: j for i, j in zip(sorted_indexes, names_list)}
n_dict
ni = [n_dict[i] for i in range(5980)]
print(ni[:5])
names_list = [f'{i:0>4}.png' for i in range(5980)]
ni = [names_list[i] for i in sorted_indexes]
df1['filename_new'] = ni

In [ ]:
df1.sort_values(by=['white_pixel_%','height','width'], ascending=False)

In [ ]:
df1['filename_new'] = ni
df1

In [ ]:
def process_image(input_path, output_path, new_width, new_height, convert_to_bw=False, expand_side='y'):
    try:
        # Чтение изображения
        image = cv2.imread(directory + '/' + input_path)

        # Конвертация в формат PNG
        _, image_png = cv2.imencode('.png', image)
        image = cv2.imdecode(image_png, cv2.IMREAD_UNCHANGED)

        # Проверка на успешное чтение изображения
        if image is None:
            raise Exception(f"Ошибка чтения изображения по пути: {input_path}")

        # Если convert_to_bw установлен в True, преобразовываем изображение в чёрно-белое
        if convert_to_bw:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Изменение размера с сохранением пропорций
        height, width = image.shape[:2]
        aspect_ratio = width / height

        # Определяем по какой стороне изменять размер
        if expand_side == 'x':
            new_height = int(new_width/aspect_ratio)
        elif expand_side == 'y':
            new_width = int(new_height*aspect_ratio)

        # Решейп и сохранение изменённого изображения
        image = cv2.resize(image, (new_width, new_height))
        cv2.imwrite(output_path, image)

    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")

In [ ]:
df1[df1['DPI_Y_A4'] < 299]

In [ ]:
output_folder = "/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/ScansClearedReshaped/"          # Конечная папка
output_folder_bad = '/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/ScansClearedReshaped/low_dpi/'   # Конечная папка для файлов с плохим качеством
# output_folder = '/content/'
filenames_dict = dict()                     # Для названий файлов
new_width = 2480                            # Ширина изображения
new_height = 3508                           # Высота изображения
convert_to_bw = True                        # Конвертировать в чёрно-белое

file_list_to_convert = list(df1[df1['DPI_Y_A4'] > 299]['Имя файла'])        # Список файлов для конвертации
files_len = len(os.listdir(directory))

for index, filename in enumerate(os.listdir(directory)):
    if filename not in file_list_to_convert:
        filenames_dict[str(index) + '.png'] = filename
        # output_path = filename[:-len(re.findall(r'\.[^.]+$', filename)[0])] + '.png' # Заменяем раcширение на .png
        output_path = str(index) + '.png' # Заменяем раcширение на .png

        if filename in file_list_to_convert:
            output_path = output_folder + output_path
        else:
            output_path = output_folder_bad + output_path

        process_image(filename, output_path, new_width, new_height, convert_to_bw, 'y')

    if index % 50 == 0:
        clear_output()
        print(f'Готово {index} из {files_len}')

In [ ]:
with open('/content/my_dict.json', 'w') as file:
    json.dump(filenames_dict, file)

In [ ]:
df1[df1['DPI_Y_A4']>=299]

In [ ]:
df1[df1['DPI_Y_A4']<299][df1['DPI_Y_A4']>=249]

In [ ]:
df1[df1['DPI_Y_A4']<249][df1['DPI_Y_A4']>=199]

In [ ]:
df1[df1['DPI_Y_A4']<199][df1['DPI_Y_A4']>=149]

In [ ]:
df1[df1['DPI_Y_A4']<149][df1['DPI_Y_A4']>=100]

In [ ]:
df1[df1['DPI_Y_A4']<100][df1['DPI_Y_A4']>=0]

In [ ]:
def process_image(input_path, output_path, new_width, new_height, convert_to_bw=False, expand_side='x'):
    try:
        # Чтение изображения
        image = cv2.imread(directory + '/' + input_path)

        # Конвертация в формат PNG
        _, image_png = cv2.imencode('.png', image)
        image = cv2.imdecode(image_png, cv2.IMREAD_UNCHANGED)

        # Проверка на успешное чтение изображения
        if image is None:
            raise Exception(f"Ошибка чтения изображения по пути: {input_path}")

        # Если convert_to_bw установлен в True, преобразовываем изображение в чёрно-белое
        if convert_to_bw:
            image = cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR) # Преобразуем в 3-канальное чёрно-белое

        # Изменение размера с сохранением пропорций
        height, width = image.shape[:2]
        aspect_ratio = width / height
        # print(f'Хотим   {new_height}x{new_width} (aspect ratio - {new_width/new_height:.3f})')
        # print(f'Есть    {height}x{width} (aspect ratio - {aspect_ratio:.3f})')
        # print(f'Получим {new_height}x{int(aspect_ratio*new_height)} (aspect ratio - {int(aspect_ratio*new_height) / new_height:.3f}) y')
        # print(f'Получим {int(new_width/aspect_ratio)}x{new_width} (aspect ratio - {new_width / int(new_width/aspect_ratio):.3f}) x')

        if expand_side == 'x':
            new_height = int(new_width/aspect_ratio)
        elif expand_side == 'y':
            new_width = int(new_height*aspect_ratio)

        image = cv2.resize(image, (new_width, new_height))

        # Сохранение изменённого изображения
        cv2.imwrite(output_path, image)

        # print(f"Изображение успешно обработано и сохранено по пути: {output_path}")
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")

# Пример использования функции:
input_path = "b9d41fe3-126b-45ec-b4b5-59f588f7e9a8.jpg" # >300 DPI
# input_path = 'invoice_263_14.12.2016.jpg'               # 200-250 dpi
output_path = "/content/" + input_path[:-len(re.findall(r'\.[^.]+$', input_path)[0])] + '.png' # Заменяем раcширение на .png
new_width = 2480
new_height = 3508
convert_to_bw = True

process_image(input_path, output_path, new_width, new_height, convert_to_bw, 'x')

Создаём батчи

In [ ]:
output_folder = "/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/ScansClearedReshaped/"

# Создаем пустой датафрейм
df2 = pd.DataFrame(columns=["Имя файла", "Ширина (px)", "Высота (px)", "DPI_X", "DPI_Y", "DPI_X_A4", "DPI_Y_A4", 'aspect_ratio'])

# Функция для получения информации о изображении
def get_image_info(file_path):
    img = Image.open(file_path)
    width, height = img.size
    dpi = img.info.get("dpi", (0, 0))
    aspect_ratio = width / height
    img.close()
    return width, height, dpi[0], dpi[1], aspect_ratio

files_len = len(os.listdir(output_folder))

# Перебираем файлы в папке
for index, filename in enumerate(os.listdir(output_folder)):
    if '.png' in filename:
        file_path = os.path.join(output_folder, filename)
        width, height, dpi_x, dpi_y, aspect_ratio = get_image_info(file_path)
        dpi_x_a4 = width / (a4_width_mm / 25.4)  # Переводим мм в дюймы
        dpi_y_a4 = height / (a4_height_mm / 25.4)  # Переводим мм в дюймы
        df2 = pd.concat([df2, pd.DataFrame({"Имя файла": [filename], "Ширина (px)": [width], "Высота (px)": [height], "DPI_X": [dpi_x], "DPI_Y": [dpi_y], "DPI_X_A4": [dpi_x_a4], "DPI_Y_A4": [dpi_y_a4], 'aspect_ratio': [aspect_ratio]})], ignore_index=True)

        if index % 200 == 0:
            clear_output()
            print(f'Готово {index} из {files_len}')

df2['DPI_X'] = df2['DPI_X'].astype(int)
df2['DPI_Y'] = df2['DPI_Y'].astype(int)
df2['Ширина (px)'] = df2['Ширина (px)'].astype(int)
df2['Высота (px)'] = df2['Высота (px)'].astype(int)

In [ ]:
df2.describe()

In [ ]:
df2.sort_values(by='aspect_ratio')

In [ ]:
batches_dict = dict()
files_per_batch = 100
batch = []

for i, filename in enumerate(list(df2.sort_values(by='aspect_ratio')['Имя файла'])):
    if (i != 0 and i % files_per_batch == 0):
        batches_dict['batch' + '_{:0>3}'.format(i // 1000) + '_{:0>3}'.format(str(i)[-3:])] = batch
        batch = []
    batch.append(filename)

    if i == len(list(df2.sort_values(by='aspect_ratio')['Имя файла'])) - 1:
        batches_dict['batch' + '_{:0>3}'.format(i // 1000) + '_{:0>3}'.format(str(i)[-3:])] = batch

In [ ]:
batches_dict.keys()

In [ ]:
import zipfile

def create_archive(file_list, archive_name):
    with zipfile.ZipFile(archive_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in file_list:
            # Получаем имя файла без полного пути
            file_name = file.split("/")[-1]
            # Добавляем файл в архив с его именем
            zipf.write(file, file_name)

# Пример использования функции:
file_list = [
    "/путь/к/файлу1.txt",
    "/путь/к/файлу2.jpg",
    "/путь/к/файлу3.pdf"
]

archive_name = "мой_архив.zip"

create_archive(file_list, archive_name)

## Просмотр изображений

In [ ]:
def compute_skew1(img1): # определние не большого поворота отностильно горизонта с помощью готового решения deskew
    grayscale_image = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    angle = determine_skew(grayscale_image)
    print(angle)
    # определили угол и повернем на него изображение
    img_rot = img1.copy()
    height, width = img_rot.shape[:2]
    # get the center coordinates of the image to create the 2D rotation matrix
    center = (width/2, height/2)
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=angle, scale=1)
    img_rot = cv2.warpAffine(src=img_rot, M=rotate_matrix, dsize=(width, height))

    return img_rot

In [ ]:
img_index = 995
image_path = directory + '/' + os.listdir(directory)[img_index]

image = cv2.imread(image_path)
img_rot = compute_skew1(image)

plt.imshow(image)
plt.axis('off')
plt.show()

plt.imshow(img_rot)
plt.axis('off')
plt.show()

In [ ]:
img_index = 995
image_path = directory + '/' + os.listdir(directory)[img_index]

img = Image.open(image_path)
plt.imshow(img)
plt.axis('off')
plt.show()

# result = is_cropped_image(image_path)
# print(result)

In [ ]:
from matplotlib.patches import Rectangle

def display_images_in_table(image_paths, num_columns=4, image_size=(6, 6), border_color='black', border_width=5, title=False, starting_index=0, show_graph=False):
    num_images = len(image_paths)
    num_rows = math.ceil(num_images / num_columns)

    fig, axes = plt.subplots(num_rows, num_columns, figsize=(num_columns * image_size[0], num_rows * image_size[1]))
    if title == True:
        fig.suptitle(title, fontsize=48, y=1.005)
    else:
        fig.suptitle(f"{starting_index}-{starting_index+50}", fontsize=48, y=1.005)

    for i, ax in enumerate(axes.flat):
        if i < num_images:
            img_path = image_paths[i]
            img = Image.open(img_path)

            # Добавляем обводку вокруг изображения
            border = Rectangle((0, 0), img.width, img.height, linewidth=border_width, edgecolor=border_color, facecolor='none')
            ax.add_patch(border)

            ax.imshow(img)
            # ax.set_title(f"Изображение {i + 1}", fontsize=24)
            ax.set_title(f"{starting_index+i}", fontsize=24)
            ax.axis('off')
        else:
            ax.axis('off')

    plt.tight_layout()
    if show_graph == True:
        plt.show()
    else:
        plt.savefig(f"/content/drive/MyDrive/imgs/{starting_index}-{starting_index+50}.png")

    plt.clf()

In [ ]:
# Пример использования:
files_list = os.listdir(directory)
image_paths = [directory + '/' + i for i in df1.sort_values(by='white_pixel_%', ascending=True)['filename']]
#image_paths = [directory + '/' + i for i in df1.sort_values(by='height', ascending=False)['filename']]
num_columns = 5  # Число столбцов в таблице (можно изменить)
border_color = 'blue'  # Цвет обводки
border_width = 5  # Ширина обводки
image_size = (6, 6)

for index in range(0, (len(files_list) // 50) + 1):
    print(index, index*50)
    display_images_in_table(image_paths[50*index:50*(index+1)], num_columns, image_size, border_color, border_width, False, index*50, True)
    if index == 4:
        break

In [ ]:
# Пример использования:
files_list = os.listdir(directory)
image_paths = [directory + '/' + i for i in df1.sort_values(by='white_pixel_%', ascending=False)['filename']]
#image_paths = [directory + '/' + i for i in df1.sort_values(by='height', ascending=False)['filename']]
num_columns = 5  # Число столбцов в таблице (можно изменить)
border_color = 'blue'  # Цвет обводки
border_width = 5  # Ширина обводки
image_size = (6, 6)

for index in range(0, (len(files_list) // 50) + 1):
    print(index, index*50)
    display_images_in_table(image_paths[50*index:50*(index+1)], num_columns, image_size, border_color, border_width, False, index*50, True)
    if index == 4:
        break

In [ ]:
# Пример использования:
files_list = os.listdir(directory)
#image_paths = [directory + '/' + i for i in df1.sort_values(by='white_pixel_%', ascending=False)['filename']]
image_paths = [directory + '/' + i for i in df1.sort_values(by='height', ascending=False)['filename']]
num_columns = 5  # Число столбцов в таблице (можно изменить)
border_color = 'blue'  # Цвет обводки
border_width = 5  # Ширина обводки
image_size = (6, 6)

for index in range(0, (len(files_list) // 50) + 1):
    print(index, index*50)
    display_images_in_table(image_paths[50*index:50*(index+1)], num_columns, image_size, border_color, border_width, False, index*50, True)
    if index == 4:
        break

In [ ]:
# Пример использования:
files_list = os.listdir(directory)
image_paths = [directory + '/' + i for i in files_list]
num_columns = 5  # Число столбцов в таблице (можно изменить)
border_color = 'blue'  # Цвет обводки
border_width = 5  # Ширина обводки
image_size = (6, 6)

for index in range(0, (len(files_list) // 50) + 1):
    print(index, index*50)
    display_images_in_table(image_paths[50*index:50*(index+1)], num_columns, image_size, border_color, border_width, False, index*50, True)
    if index == 4:
        break

In [ ]:
# Пример использования:
image_paths = [directory + '/' + i for i in list(df[df['Ориентация'] == 0]['Название файла'])]
num_columns = 5  # Число столбцов в таблице (можно изменить)
border_color = 'blue'  # Цвет обводки
border_width = 5  # Ширина обводки
image_size = (8, 8)

display_images_in_table(image_paths, num_columns, image_size, border_color, border_width)

In [ ]:
image_paths = [directory + '/' + i for i in list(df[df['Обрезан'] == 1]['Название файла'])]
num_columns = 5  # Число столбцов в таблице (можно изменить)
border_color = 'blue'  # Цвет обводки
border_width = 5  # Ширина обводки
image_size = (8, 8)

display_images_in_table(image_paths, num_columns, image_size, border_color, border_width, 'Все обрезанные (дефектные) изображения')

In [ ]:
img = image.load_img(directory + '/' + random.choice(os.listdir(directory)))
# image.img_to_array(img)

In [ ]:
img = image.load_img(directory + '/' + '2022_0_12_17_18_8_e67afc64-fd6c-4e50-9686-79bdc95f3818.jpeg')
plt.figure(figsize=(14, 14))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
img_index = 0
img = image.load_img(directory + '/' + os.listdir(directory)[img_index])
plt.figure(figsize=(14, 14))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# Пример использования:
# image_paths = [directory + '/' + i for i in list(df[500:700]['Название файла'])]
files_list = os.listdir(directory)
image_paths = [directory + '/' + i for i in files_list[1250:1500]]
num_columns = 5  # Число столбцов в таблице (можно изменить)
border_color = 'blue'  # Цвет обводки
border_width = 5  # Ширина обводки
image_size = (5, 5)

display_images_in_table(image_paths, num_columns, image_size, border_color, border_width)

In [ ]:
df[:200]

# Модель поворота изображений

## *Подготовка данных*

In [ ]:
# Создаём выборку только из вертикальных изображений
files_dirs = []
# dont_load_index = [i-1 for i in [184, 185, 171, 157, 129, 12]] + [i+200-1 for i in [162, 136, 70, 48]] + [i+400-1 for i in [78,43]]
# dont_load_index = [i-1 for i in [184, 185, 171, 157, 129, 12]] + [i+200-1 for i in [162, 136, 70, 48]] + [i+400-1 for i in [78,43]] + [i+500-1 for i in [39, 27, 25]] + [i+700-1 for i in [193,79,55]] + [i+900-1 for i in [83, 60]]
dont_load_index = [1000-1 + i for i in [246,241,240,238,208,175,114,92,85,78,51,48,46,42,40,35,1]] # + [1250-1 + i for i in [245,240,188,178,135,102]]

files_list = os.listdir(directory)
for i in range(1000,1500):
    if i not in dont_load_index:
        files_dirs.append(directory + '/' + files_list[i])

print(files_dirs[0])
print(len(files_dirs))

In [ ]:
# Размер, к которому вы хотите привести изображения
target_size = (224, 224)

image_list = []
for filename in files_dirs:
    image = Image.open(filename)
    image = image.resize(target_size)  # Изменение размера
    # image = np.array(image) / 255.0
    image_list.append(image)

# Поворот изображений и добавление их в общую выборку
rotated_images = []
for image in image_list:
    original_image = image.copy()
    rotated_images.append(original_image)  # Оригинальное изображение

    rotated_90 = image.rotate(90)
    rotated_images.append(rotated_90)  # Поворот на 90°

    rotated_minus_90 = image.rotate(-90)
    rotated_images.append(rotated_minus_90)  # Поворот на -90°

    rotated_180 = image.rotate(180)
    rotated_images.append(rotated_180)  # Поворот на 180°

del image_list

# Перевод в numpy массив и нормализация (диапазон [0, 1])
rotated_images = [np.array(image) / 255.0 for image in rotated_images]
rotated_images = [i for i in rotated_images if i.shape == (224, 224, 3)]
rotated_images = np.array(rotated_images)

# Разметка изображений
labels = [0, 1, 2, 3] * (len(rotated_images) // 4)

# Перевод меток в формат One-Hot Encoding
one_hot_labels = np.array(one_hot(labels, depth=4))

In [ ]:
# Сохранить данные
np.save('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/one_hot_labels.npy', one_hot_labels)
np.save('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/rotated_images.npy', rotated_images)

In [ ]:
# Сохранить данные
np.save('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/one_hot_labels_l.npy', one_hot_labels)
np.save('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/rotated_images_l.npy', rotated_images)

In [ ]:
# Загрузить данные
one_hot_labels = np.load('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/one_hot_labels.npy')
rotated_images = np.load('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/rotated_images.npy')

In [ ]:
# Загрузить данные
one_hot_labels = np.load('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/one_hot_labels_l.npy')
rotated_images = np.load('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/rotated_images_l.npy')

In [ ]:
# Создание обучающей и тестовой выборок
xTrain, xTest, yTrain, yTest = train_test_split(rotated_images, one_hot_labels, test_size=0.2, random_state=42)
del one_hot_labels, rotated_images

In [ ]:
x = np.vstack((xTrain, xTest))
y = np.vstack((yTrain, yTest))

In [ ]:
import gc
gc.collect()

## *Модель MobileNetV3Small*
> Лучшая точность 98.72%

In [ ]:
def get_MobileNetV3Small_model():
    # Загрузка предобученной модели MobileNetV3-small без верхних слоев
    base_model = MobileNetV3Small(weights='imagenet', include_top=False)

    # Добавление своих верхних слоев для многоклассовой классификации
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)                # Полносвязный слой
    x = Dense(64, activation='relu')(x)                 # Полносвязный слой
    predictions = Dense(4, activation='softmax')(x)     # 4 - количество классов

    model = Model(inputs=base_model.input, outputs=predictions)

    # Замораживаем слои базовой модели
    for layer in base_model.layers:
        layer.trainable = False

    # Компиляция модели
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def get_MobileNetV3Small_model_2():
    # Загрузка предобученной модели MobileNetV3-small без верхних слоев
    base_model = MobileNetV3Small(weights='imagenet', include_top=False)

    # Добавление своих верхних слоев для многоклассовой классификации
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)                # Полносвязный слой
    x = Dense(128, activation='relu')(x)                 # Полносвязный слой
    x = Dense(64, activation='relu')(x)                 # Полносвязный слой
    predictions = Dense(4, activation='softmax')(x)     # 4 - количество классов

    model = Model(inputs=base_model.input, outputs=predictions)

    # Замораживаем слои базовой модели
    for layer in base_model.layers:
        layer.trainable = False

    # Компиляция модели
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def get_MobileNetV3Small_model_2():
    # Загрузка предобученной модели MobileNetV3-small без верхних слоев
    base_model = MobileNetV3Small(weights='imagenet', include_top=False)

    # Добавление своих верхних слоев для многоклассовой классификации
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)                # Полносвязный слой
    x = Dense(128, activation='relu')(x)                 # Полносвязный слой
    x = Dense(64, activation='relu')(x)                 # Полносвязный слой
    predictions = Dense(4, activation='softmax')(x)     # 4 - количество классов

    model = Model(inputs=base_model.input, outputs=predictions)

    # Замораживаем слои базовой модели
    for layer in base_model.layers:
        layer.trainable = False

    # Компиляция модели
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def print_model_acc(model):
    # Оценка модели на тестовой выборке
    evaluation = model.evaluate(xTest, yTest)
    accuracy = evaluation[1]
    print(f"Точность модели на тестовой выборке: {accuracy * 100:.2f}%")

# Для сбора мусора
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

def print_train_graph(history):
    # Построение графика обучения
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

### Обучение модель
> Лучшая точность 98.72%

In [ ]:
model = get_MobileNetV3Small_model()
# model.compile(optimizer=Adam(learning_rate=0.00025), loss='categorical_crossentropy', metrics=['accuracy'])

# Создание объекта ModelCheckpoint для сохранения весов
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max')
garbage_collector = GarbageCollectorCallback()

# # Загрузка лучших весов после обучения
# model.load_weights("best_model.h5")

# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=32, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model.h5")

print_train_graph(history)

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=32, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model.h5")

print_train_graph(history)

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=32, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model.h5")

print_train_graph(history)

In [ ]:
def train_model(model):
    # Обучение модели с использованием ModelCheckpoint
    history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=32, callbacks=[checkpoint, garbage_collector])

    # Загрузка лучших весов после обучения
    model.load_weights("best_model.h5")

    # Построение графика обучения
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    return model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=32, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model.h5")

print_train_graph(history)

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=64, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model.h5")

print_train_graph(history)

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=72, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model.h5")

print_train_graph(history)

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=100, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model.h5")

print_train_graph(history)

In [ ]:
# Оценка модели на тестовой выборке
evaluation = model.evaluate(xTest, yTest)

# Оценка возвращает список метрик, установленных при компиляции модели
# В данном случае, мы используем метрику 'accuracy'

accuracy = evaluation[1]
print(f"Точность модели на тестовой выборке: {accuracy * 100:.2f}%")

### Обучение более мощьной модели
> Лучшая точность 98.47%

In [ ]:
model = get_MobileNetV3Small_model_2()

# Создание объекта ModelCheckpoint для сохранения весов
checkpoint = ModelCheckpoint("best_model_v2.h5", monitor='val_accuracy', save_best_only=True, mode='max')
garbage_collector = GarbageCollectorCallback()

# # Загрузка лучших весов после обучения
# model.load_weights("best_model_v2.h5")

# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=64, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model_v2.h5")

print_train_graph(history)

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=32, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model_v2.h5")

print_train_graph(history)

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=32, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model_v2.h5")

print_train_graph(history)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])                                  # Компиляция модели
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=32, callbacks=[checkpoint, garbage_collector])    # Обучение модели
model.load_weights("best_model_v2.h5")                                                                                                      # Загрузка весов
print_train_graph(history)                                                                                                                  # График

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])                                  # Компиляция модели
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=32, callbacks=[checkpoint, garbage_collector])    # Обучение модели
model.load_weights("best_model_v2.h5")                                                                                                      # Загрузка весов
print_train_graph(history)                                                                                                                  # График

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.00025), loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=10, batch_size=32, callbacks=[checkpoint, garbage_collector])

# Загрузка лучших весов после обучения
model.load_weights("best_model.h5")

print_train_graph(history)

In [ ]:
print_model_acc(model)

## *Обучение архитектуры autokeras*
> Лучшая точность 99.49%

> **Валидационная выборка 99.64%**

In [ ]:
pip install autokeras

In [ ]:
import autokeras as ak

# Создайте экземпляр класса ImageClassifier
clf = ak.ImageClassifier(max_trials=1)  # Максимальное количество попыток

# Обучите модель
clf.fit(xTrain, yTrain, epochs=3)  # x_train - обучающие изображения, y_train - метки классов

In [ ]:
import autokeras as ak

# Создаём ImageClassifier
clf = ak.ImageClassifier(max_trials=1)

# Обучаем модель
clf.fit(xTrain, yTrain, epochs=5)

# Результаты модели
results = clf.evaluate(xTest, yTest)
print(results)

# Экспортируем модель
best_model = clf.export_model()
best_model.summary()

# Сохраняем модель
best_model.save('best_model_auto.h5')

In [ ]:
import tensorflow as tf

def create_custom_neural_network(input_shape, num_classes, block_type="vanilla", normalize=True, augment=False,
                                kernel_size=3, num_blocks=1, num_layers=2, max_pooling=True, separable=False,
                                dropout=0.25, filters=(32, 64), reduction_type="flatten", dropout_head=0.5,
                                optimizer='adam', learning_rate=0.001):
    # Создаем входной слой
    input_layer = tf.keras.layers.Input(shape=input_shape)

    # Подсистема изображения
    x = input_layer
    if block_type == "vanilla":
        if normalize:
            x = tf.keras.layers.BatchNormalization()(x)
        if augment:
            x = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")(x)
            x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)(x)
        for _ in range(num_blocks):
            for _ in range(num_layers):
                x = tf.keras.layers.Conv2D(filters[0], kernel_size, padding='same', activation='relu')(x)
                if max_pooling:
                    x = tf.keras.layers.MaxPooling2D()(x)
                if separable:
                    x = tf.keras.layers.SeparableConv2D(filters[1], kernel_size, padding='same', activation='relu')(x)
                else:
                    x = tf.keras.layers.Conv2D(filters[1], kernel_size, padding='same', activation='relu')(x)
                x = tf.keras.layers.BatchNormalization()(x)
                x = tf.keras.layers.Dropout(dropout)(x)

    # Голова классификации
    if reduction_type == "flatten":
        x = tf.keras.layers.Flatten()(x)
    else:
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(dropout_head)(x)
    output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    # Создаем модель
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    # Компилируем модель с выбранным оптимизатором и скоростью обучения
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Пример использования функции для создания модели
input_shape = (224, 224, 3)  # Форма входных изображений
num_classes = 4  # Количество классов
model = create_custom_neural_network(input_shape, num_classes)
model.summary()

In [ ]:
def print_model_acc(model):
    # Оценка модели на тестовой выборке
    evaluation = model.evaluate(xTest, yTest)
    accuracy = evaluation[1]
    print(f"Точность модели на тестовой выборке: {accuracy * 100:.2f}%")

# Для сбора мусора
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

def print_train_graph(history):
    # Построение графика обучения
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Создание объекта ModelCheckpoint для сохранения весов
checkpoint = ModelCheckpoint("best_model_auto.h5", monitor='val_accuracy', save_best_only=True, mode='max')
garbage_collector = GarbageCollectorCallback()

# # Загрузка лучших весов после обучения
# model.load_weights("best_model_auto.h5")

# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=5, batch_size=32, callbacks=[checkpoint])

# Загрузка лучших весов после обучения
model.load_weights("best_model_auto.h5")

print_train_graph(history)

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=5, batch_size=32, callbacks=[checkpoint])

# Загрузка лучших весов после обучения
model.load_weights("best_model_auto.h5")

print_train_graph(history)

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=5, batch_size=32, callbacks=[checkpoint])

# Загрузка лучших весов после обучения
model.load_weights("best_model_auto.h5")

print_train_graph(history)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=5, batch_size=32, callbacks=[checkpoint])
model.load_weights("best_model_auto.h5")
print_train_graph(history)

In [ ]:
# Оценка модели на тестовой выборке
evaluation = model.evaluate(xTest, yTest)

accuracy = evaluation[1]
print(f"Точность модели на тестовой выборке: {accuracy * 100:.2f}%")

##### **Валидационная выборка**

In [ ]:
from keras.models import load_model

# Загрузка модели
model = load_model('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/my_model.h5')

In [ ]:
# Оценка модели на проверочной выборке
evaluation = model.evaluate(x, y)

accuracy = evaluation[1]
print(f"Точность модели на проверочной выборке: {accuracy * 100:.2f}%")

In [ ]:
# Используйте метод predict для модели, чтобы получить предсказания
predictions = model.predict(prepare_img(files_dirs[0]))  # Предполагается, что ваша модель ожидает 2D массив

# Результат будет в переменной predictions, которая будет содержать предсказанные значения
print("Предсказанные значения:", predictions)

## *Проверка скорости работы лучшей модели*
> Проверяем, сколько каждого типа изображений в базе по менению сети

> **Средняя скорость обработки одного изображения - 57.75 мс.**

In [ ]:
files_dirs = [directory + '/' + i for i in os.listdir(directory)]

In [ ]:
def open_img(img_path):
    return Image.open(img_path)


def prepare_img(img_path):
    image = Image.open(img_path)
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])

    return image


def load_img(img_path, single=False):
    # Загрузка изображения и изменение размера, для модели поворота изображений
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0

    if single == True:
        img_array = np.expand_dims(img_array, axis=0)

    return img_array


def load_img_cv2(img_path, single=False):
    # Загрузка изображения и изменение размера, для модели поворота изображений
    image = cv2.imread(img_path)
    if len(image.shape) != 3 or image.shape[2] != 3:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image = cv2.resize(image, (224, 224))
    image_array = np.array(image) / 255.0

    if single==True:
        image_array = np.expand_dims(image_array, axis=0)

    return image_array


# def prepare_imgs(img_paths):
#     return np.array([np.array(Image.open(img_path).resize((224, 224))) / 255.0 for img_path in img_paths])


def prepare_imgs(img_paths):
    return np.array([load_img_cv2(img_path) for img_path in img_paths])


def rotate_img_by_index(img, index):
    if index == 0:
        return img
    return img.rotate([0,90,-90,180][index])

In [ ]:
def rotate_img_by_index(img, index):
    if index == 0:
        return img
    return img.rotate([0,90,-90,180][index])

In [ ]:
prepare_img(files_dirs[0]).shape

In [ ]:
from keras.models import load_model

# Загрузка модели
model = load_model('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/my_model.h5')

In [ ]:
wrong_list = sorted([i-1 for i in [184, 185, 171, 157, 129, 12]] + [i+200-1 for i in [162, 136, 70, 48]] + [i+400-1 for i in [78,43]] + [i+500-1 for i in [39, 27, 25]] + [i+700-1 for i in [193,79,55]] + [i+900-1 for i in [83, 60]] + [1000-1 + i for i in [246,241,240,238,208,175,114,92,85,78,51,48,46,42,40,35,1]] + [1250-1 + i for i in [245,240,188,178,135,102]])

In [ ]:
max_files = 50
print(f'Всего файлов: {len(files_dirs[:max_files])}')

# Предсказываем значения
with timex():
    predictions = model.predict(prepare_imgs(files_dirs[:max_files]))

# print("Предсказанные значения:", predictions)

In [ ]:
print(f'Всего файлов: {len(files_dirs[100:200])}')

# Предсказываем значения
x = prepare_imgs(files_dirs[100:200])
with timex():
    predictions = model.predict(x)

In [ ]:
print(f'Всего файлов: {len(files_dirs[200:500])}')

# Предсказываем значения
x = prepare_imgs(files_dirs[200:500])
with timex():
    predictions = model.predict(x)

In [ ]:
print(f'Всего файлов: {len(files_dirs[500:1000])}')

# Предсказываем значения
x = prepare_imgs(files_dirs[500:1000])
with timex():
    predictions = model.predict(x)

In [ ]:
((5.69 / 100) + (21.85 / 300) + (21.76 / 500)) / 3

In [ ]:
0.05775 * 1000

##### Предсказание классов

In [ ]:
print(f'Всего файлов: {len(files_dirs)}')

# Предсказываем значения
with timex():
    predictions = model.predict(prepare_imgs(files_dirs))

In [ ]:
indices = np.argmax(predictions, axis=1)
print(indices)

In [ ]:
from collections import Counter

my_list = [1, 2, 2, 3, 3, 3, 4, 4, 4, 4]
element_count = Counter(my_list)

for element, count in element_count.items():
    print(f"{element}: {count}")

# Регулярные выражения

In [ ]:
import re

In [ ]:
text_samples = ['fsad321;klj;jskad 34 jkdsflalsfdalkjsfd 93421809 dshfakjhlksad',
                'Поставщик: Адрес:043289, Республика Казахстан, Алматы, Бродского, Товарищество с ограниченной ответсвтенностью "Ресми-Табыс", БИН:991240008188',
                'Поставщик: Адрес:043289, Республика Казахстан, Алматы, Бродского, Товарищество с ограниченной ответсвтенностью "Ресми-Табыс", БИН:991240008188одоыю..']

patterns_list = [r'\d\d+',      # Любые 2+ цирфы
                 r'\d\d+$',     # Любые 2+ цифры в конце строки
                 r'\d{12}',     # БИН - 12 цифр
                 r'\b\d\d+\b'   # Любые 2+ цифры с границами слов (слева и справа должны быть не буквенные символы)
                 ]

for text in text_samples:
    print('Пример текста:', text)
    for pattern in patterns_list:
        print(f'Паттерн {pattern}: {re.findall(pattern, text)}')
    print()

# Преобразования изображений

In [ ]:
file_list = os.listdir(directory) # Список файлов
file_paths_list = [directory + '/' + i for i in file_list]

In [ ]:
import cv2
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np

from google.colab.patches import cv2_imshow

def transform_image(image_path):
    # Загрузка изображения с использованием Pillow
    image = Image.open(image_path)

    # Преобразование изображения в формат NumPy
    image_np = np.array(image)

    # Уменьшение шума (фильтрация шума и бинаризация)
    denoised_image = cv2.fastNlMeansDenoisingColored(image_np, None, 10, 10, 7, 21)
    gray_image = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Улучшение контраста (адаптивная гистограммная эквализация)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(gray_image)

    # Выделение границ (детектор Canny)
    edges = cv2.Canny(gray_image, 100, 200)

    # Размытие (размытие Гаусса)
    blurred_image = cv2.GaussianBlur(denoised_image, (5, 5), 0)

    # Улучшение разрешения (увеличение с помощью интерполяции)
    upscaled_image = cv2.resize(image_np, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # Отображение результатов
    cv2_imshow('Denoised and Binary Image', binary_image)
    cv2_imshow('Enhanced Image', enhanced_image)
    cv2_imshow('Edge Detected Image', edges)
    cv2_imshow('Blurred Image', blurred_image)
    cv2_imshow('Upscaled Image', upscaled_image)

    # Ожидание нажатия клавиши для закрытия окон
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
def transform_image(image_path):
    # Загрузка изображения с использованием Pillow
    image = Image.open(image_path)

    # Преобразование изображения в формат NumPy
    image_np = np.array(image)

    # Уменьшение шума (фильтрация шума и бинаризация)
    denoised_image = cv2.fastNlMeansDenoisingColored(image_np, None, 10, 10, 7, 21)
    gray_image = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Улучшение контраста (адаптивная гистограммная эквализация)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(gray_image)

    # Выделение границ (детектор Canny)
    edges = cv2.Canny(gray_image, 100, 200)

    # Размытие (размытие Гаусса)
    blurred_image = cv2.GaussianBlur(denoised_image, (5, 5), 0)

    # Улучшение разрешения (увеличение с помощью интерполяции)
    upscaled_image = cv2.resize(image_np, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # Отображение результатов с использованием matplotlib
    plt.figure(figsize=(32, 28))
    fs = 48

    plt.subplot(2, 3, 1)
    plt.imshow(image)
    plt.title('Original', fontsize=fs)
    plt.axis('off')

    plt.subplot(2, 3, 2)
    plt.imshow(binary_image, cmap='gray')
    plt.title('Denoised + Binary', fontsize=fs)
    plt.axis('off')

    plt.subplot(2, 3, 3)
    plt.imshow(enhanced_image, cmap='gray')
    plt.title('Enhanced', fontsize=fs)
    plt.axis('off')

    plt.subplot(2, 3, 4)
    plt.imshow(edges, cmap='gray')
    plt.title('Edges', fontsize=fs)
    plt.axis('off')

    plt.subplot(2, 3, 5)
    plt.imshow(blurred_image)
    plt.title('Blurred', fontsize=fs)
    plt.axis('off')

    plt.subplot(2, 3, 6)
    plt.imshow(upscaled_image)
    plt.title('Upscaled', fontsize=fs)
    plt.axis('off')

    plt.show()

In [ ]:
# Результат функции
transform_image(file_paths_list[0])

# Остальное

## Perspective Transformation

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_homography_matrix(source, destination):
    """ Calculates the entries of the Homography matrix between two sets of matching points.

    Args
    ----
        - `source`: Source points where each point is int (x, y) format.
        - `destination`: Destination points where each point is int (x, y) format.

    Returns
    ----
        - A numpy array of shape (3, 3) representing the Homography matrix.

    Raises
    ----
        - `source` and `destination` is lew than four points.
        - `source` and `destination` is of different size.
    """
    assert len(source) >= 4, "must provide more than 4 source points"
    assert len(destination) >= 4, "must provide more than 4 destination points"
    assert len(source) == len(destination), "source and destination must be of equal length"
    A = []
    b = []
    for i in range(len(source)):
        s_x, s_y = source[i]
        d_x, d_y = destination[i]
        A.append([s_x, s_y, 1, 0, 0, 0, (-d_x)*(s_x), (-d_x)*(s_y)])
        A.append([0, 0, 0, s_x, s_y, 1, (-d_y)*(s_x), (-d_y)*(s_y)])
        b += [d_x, d_y]
    A = np.array(A)
    h = np.linalg.lstsq(A, b)[0]
    h = np.concatenate((h, [1]), axis=-1)
    return np.reshape(h, (3, 3))


if __name__ == "__main__":
    source_points = np.array([
        [200, 541],
        [1216, 413],
        [1373, 993],
        [267, 1208]
    ])
    destination_points = np.array([
        [0, 0],
        [300, 0],
        [300, 300],
        [0, 300],
    ])
    source_image = cv2.imread("sample.jpeg")
    t_source_image = source_image.copy()

    # draw markings on the source image
    for i, pts in enumerate(source_points):
        cv2.putText(source_image, str(i+1), (pts[0] + 15, pts[1]), cv2.FONT_HERSHEY_PLAIN, 3, (0, 215, 255), 5)
        cv2.circle(source_image, pts, 10, (0, 215, 255), 10)

    h = get_homography_matrix(source_points, destination_points)
    destination_image = cv2.warpPerspective(t_source_image, h, (300, 300))

    figure = plt.figure(figsize=(12, 6))

    subplot1 = figure.add_subplot(1, 2, 1)
    subplot1.title.set_text("Source Image")
    subplot1.imshow(cv2.cvtColor(source_image, cv2.COLOR_BGR2RGB))

    subplot2 = figure.add_subplot(1, 2, 2)
    subplot2.title.set_text("Destination Image")
    subplot2.imshow(cv2.cvtColor(destination_image, cv2.COLOR_BGR2RGB))

    # plt.show()
    plt.savefig("output.png")

## tabula

In [ ]:
pip install Pillow

In [ ]:
pip install reportlab

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image
from PIL import Image as PILImage

# Список файлов изображений, которые вы хотите включить в PDF
img_index = 0
image_files = [directory + '/' + os.listdir(directory)[img_index]]

# Имя выходного PDF-файла
output_pdf = "output.pdf"

# Создаем объект PDF-документа
doc = SimpleDocTemplate(output_pdf, pagesize=letter)

# Создаем список элементов для добавления в PDF
elements = []

# Максимальные размеры страницы
max_width, max_height = letter

# Добавляем каждое изображение в PDF
for image_file in image_files:
    img = PILImage.open(image_file)
    width, height = img.size

    # Масштабируем изображение по размеру страницы, сохраняя соотношение сторон
    if width > max_width or height > max_height:
        aspect_ratio = width / height
        if aspect_ratio >= 1:
            new_width = max_width
            new_height = max_width / aspect_ratio
        else:
            new_height = max_height
            new_width = max_height * aspect_ratio
        img.thumbnail((new_width, new_height), PILImage.ANTIALIAS)

    image = Image(image_file, width=new_width, height=new_height)
    elements.append(image)

# Строим документ
doc.build(elements)


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image, Spacer, KeepTogether

# Список файлов изображений, которые вы хотите включить в PDF
img_index = 0
image_files = [directory + '/' + os.listdir(directory)[img_index]]
# image_files = ["image1.jpg", "image2.jpeg", "image3.png"]

# Имя выходного PDF-файла
output_pdf = "output.pdf"

# Создаем объект PDF-документа
doc = SimpleDocTemplate(output_pdf, pagesize=letter)

# Создаем список элементов для добавления в PDF
elements = []

# Добавляем каждое изображение в PDF
for image_file in image_files:
    image = Image(image_file, width=letter[0], height=letter[1])
    # elements.append(KeepTogether(Spacer(width=letter[0], height=letter[1])))
    elements.append(image)



# Строим документ
doc.build(elements)

In [ ]:
import tabula

# Указываем путь к PDF-файлу с таблицей
file_path = "путь_к_файлу.pdf"

# Извлекаем таблицу из PDF
df = tabula.read_pdf(file_path, pages='all')

# Далее вы можете обработать извлеченные данные с помощью pandas или других библиотек


# Информация о базе

## **База**
---
### *Общая информация*
* В базе данных предоставлены файлы 3 типов: .jpg, .jpeg и .png . Если нужно будет обрабатывать файлы .pdf, то нужно будет преобразовать файлы.

* Есть несколько типов документов

* В основном база данных представляет сканы документов, но есть и несколько фото.

* Есть фотографии с дефектами, разным углом поворота.

---
### *Данные*
* Язык данных: en, rus, есть редкое упоминание казахского языка

## Этапы обработки изображений
---
1. Модель классификации. Определяет тип переданного файла (.png, .jpg или .pdf) (может потребоватся находить нужную страницу в документе .pdf)
2. Модель определения угла поворота. Некоторые документы могут быть повёрнуты на -90, 90 или 180 градусов.
3. Модель сегментации. Определение на документе расположения всех полей.
4. Character‑Region Awareness For Text detection (CRAFT). Поля, в которых содержится много текста, предварительно разбиваются на отдельные слова.
5. Модель распознавания текста. Из переданного изображения извлекается текст.
6. Постпроцессинг ответа. Исправление опечаток, ошибок распознавания, сверка со справочниками.

# Обновление датафрейма

In [ ]:
4422.png 5089.png 5717.png

In [ ]:
df2 = pd.read_excel('1.xlsx')
df2

In [ ]:
df2 = df2.fillna(0)

In [ ]:
df2 = df2.astype('int32')

In [ ]:
df2

In [ ]:
names_dict = {
        'индекс файла':               'file_index',
        'грязь':                        'dirt',
        'грязь за пределами документа': 'dirt_out',
        'плохая печать':                'bad_print',
        'нестандартный контраст':       'ns_contrast',
        'искажение текста':             'distortion_text',
        'искажение документа':          'distortion_document',
        'точечный текст':               'dotted_text',
        'размытый текст':               'blurry_text',
        'выброс':                       'outlier',
        'отсутствующие поля':           'missing_fields',
        'пустые поля':                  'blank_fields',
        'рукописный текст':             'handwritten_text',
        'нестандартное форматирование текста':  'ns_text_formatting',
        'нестандартное расположение текста':    'ns_text_arrangement',
        'нестандартный якорь поля':             'ns_text_anchor',
        'нестандартная верхняя таблица':         'ns_top_table',
        'нестандартная нижняя таблица':         'ns_bottom_table',
        'обрезанные границы таблиц':    'blank_boundaries',
        'обрезанный текст':             'cropped_text',
        'проверка таблиц':              'checking_table',
        '2 документа':                  '2_documents',
        'казахский язык':               'kazakh_lang',
        '2 банка в верхней таблице':    '2_bank_accounts',
        'не белый фон':                 'not_white_background',
        'обрезанный контент':           'cropped_content'}

In [ ]:
order_list = [int(i[:-4]) for i in list(df1['filename_new'])]

In [ ]:
df2 = df2.iloc[order_list]

In [ ]:
df2 = df2.reset_index()

In [ ]:
df2.drop('index', axis=1, inplace=True)

In [ ]:
df_new = pd.concat([df1,df2], axis=1)
df_new

In [ ]:
df_new = df_new.rename({'название файла':'индекс файла'})

In [ ]:
df_new_en = df_new.rename(names_dict)

In [ ]:
df_new.columns

In [ ]:
df_new.drop('defect', axis=1, inplace=True)

In [ ]:
# Сохранение
df_new.to_csv('/content/drive/MyDrive/df_info_new_ru.csv', index=False)

In [ ]:
# Сохранение
df_new_en.to_csv('/content/drive/MyDrive/df_info_new_en.csv', index=False)

# Анализ дефектов

In [ ]:
df3 = df_new[['индекс файла', 'грязь', 'грязь за пределами документа', 'плохая печать',
       'нестандартный контраст', 'искажение текста', 'искажение документа',
       'точечный текст', 'размытый текст', 'выброс', 'отсутствующие поля',
       'пустые поля', 'рукописный текст',
       'нестандартное форматирование текста',
       'нестандартное расположение текста', 'нестандартный якорь поля',
       'нестандартная верхняя таблица', 'нестандартная нижняя таблица',
       'обрезанные границы таблиц', 'обрезанный текст', 'проверка таблиц',
       '2 документа', 'казахский язык', '2 банка в верхней таблице',
       'не белый фон', 'обрезанный контент']]

df3

In [ ]:
df3 = df3.sort_values(by=['индекс файла'])

In [ ]:
df3.reset_index(drop=True, inplace=True)

In [ ]:
df3.drop('индекс файла', inplace=True, axis=1)

In [ ]:
# Определение функции для суммирования каждых 100 строк
def sum_every_100_rows(df):
    result = pd.DataFrame()
    for i in range(0, len(df), 100):
        subset = df.iloc[i:i+100]
        result = pd.concat([result, pd.DataFrame([subset.sum()])], ignore_index=True)

    return result

# Применение функции к df3
result_df = sum_every_100_rows(df3)

# Вывод результата
result_df

In [ ]:
result_df.describe()

In [ ]:
# Транспонируем датафрейм для удобства построения графика
df = pd.DataFrame(result_df.sum()).T

# Транспонируем DataFrame для удобства построения графика
df = df.transpose()

# Рассчитываем процентные значения
df['Percentage'] = df[0] / 5980 * 100

# Строим столбчатый график
plt.figure(figsize=(10, 15))
sns.barplot(x=df.index, y=df[0], palette=sns.color_palette("light:salmon_r", 25))
plt.title('Статистика по новым колонкам')
plt.xlabel('Колонки')
plt.ylabel('Количество')
plt.xticks(rotation=90)  # Повернем подписи по оси X для лучшей читаемости

# Добавим аннотации с процентами
for i, count in enumerate(df[0]):
    percentage = df.loc[df.index[i], 'Percentage']
    plt.text(i, count + 10, f"{count}\n{percentage:.2f}%", ha='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
result_df['Всего ошибок'] = result_df.sum(axis=1)

In [ ]:
result_df.drop('Всего ошибок', axis=1, inplace=True)

In [ ]:
# Добавляем столбец с процентами
df_percent = result_df.div(result_df.sum(axis=1), axis=0) * 100

# Создаем тепловую карту
plt.figure(figsize=(20, 40))
sns.heatmap(result_df, annot=result_df.applymap(lambda x: f'{x}' if x != 0 else ''), fmt='', cmap="mako_r", cbar_kws={'label': 'Count', 'aspect': 50}, annot_kws={"size": 15}, linewidth=.5)
plt.title('Статистика по новым колонкам', fontsize=20)
plt.show()

# Тепловая карта с процентами
plt.figure(figsize=(20, 40))
sns.heatmap(df_percent, annot=df_percent.applymap(lambda x: f'{x:.1f}' if x != 0 else ''), fmt='', cmap=sns.color_palette("light:salmon", as_cmap=True), cbar_kws={'label': 'Percentage', 'aspect': 50}, annot_kws={"size": 15}, linewidth=.5)
plt.title('Статистика по новым колонкам', fontsize=20)
plt.show()

# Тепловая карта с процентами и подписями только для ненулевых значений
plt.figure(figsize=(20, 40))
sns.heatmap(df_percent, annot=df_percent.applymap(lambda x: f'{x:.1f}' if x != 0 else ''), fmt='', cmap='YlGnBu', cbar_kws={'label': 'Percentage', 'aspect': 50}, annot_kws={"size": 15}, linewidth=.5)
plt.title('Errors Percentage Heatmap')
plt.show()

In [ ]:
def plot_heatmap(df, start_i, num_rows, percent=False, true_order=False, fig_all=False):
    # График для первых 30 строк
    if fig_all == False:
        plt.figure(figsize=(20, 20))
    else:
        plt.figure(figsize=(20, 40))

    if percent == False:
        if true_order == False:
            sns.heatmap(result_df.iloc[start_i:start_i+num_rows, :], annot=result_df.iloc[start_i:start_i+num_rows, :].applymap(lambda x: f'{x}' if x != 0 else ''), fmt='', cmap="mako_r", cbar_kws={'label': 'Количество', 'aspect': 50}, annot_kws={"size": 15}, linewidth=.5)
        else:
            sns.heatmap(result_df.iloc[list(result_df.sort_values(by=['Всего ошибок']).index)[start_i:start_i+num_rows]], annot=result_df.iloc[list(result_df.sort_values(by=['Всего ошибок']).index)[start_i:start_i+num_rows]].applymap(lambda x: f'{x}' if x != 0 else ''), fmt='', cmap="mako_r", cbar_kws={'label': 'Количество', 'aspect': 50}, annot_kws={"size": 15}, linewidth=.5)
    else:
        df_percent = result_df.div(result_df.sum(axis=1), axis=0) * 100
        if true_order == False:
            sns.heatmap(df_percent.iloc[start_i:start_i+num_rows, :], annot=df_percent.iloc[start_i:start_i+num_rows, :].applymap(lambda x: f'{x:.1f}' if x != 0 else ''), fmt='', cmap='YlGnBu', cbar_kws={'label': 'Проценты', 'aspect': 50}, annot_kws={"size": 15}, linewidth=.5)
        else:
            sns.heatmap(df_percent.iloc[list(result_df.sort_values(by=['Всего ошибок']).index)[start_i:start_i+num_rows]], annot=df_percent.iloc[list(result_df.sort_values(by=['Всего ошибок']).index)[start_i:start_i+num_rows]].applymap(lambda x: f'{x:.1f}' if x != 0 else ''), fmt='', cmap='YlGnBu', cbar_kws={'label': 'Проценты', 'aspect': 50}, annot_kws={"size": 15}, linewidth=.5)

    plt.title(f'Статистика по новым столбцам (Батчи {start_i}-{start_i+num_rows})')
    plt.show()

In [ ]:
def plot_heatmap(df, start_row, num_rows):
    # Проверяем, что начальная строка не превышает общее количество строк в датафрейме
    start_row = min(start_row, df.shape[0] - 1)

    # Выбираем строки для отображения
    selected_rows = df.iloc[start_row:start_row + num_rows, :]

    # Добавляем столбец с процентами
    df_percent = selected_rows.div(selected_rows.sum(axis=1), axis=0) * 100

    # Строим тепловую карту
    plt.figure(figsize=(10, 6))
    sns.heatmap(selected_rows, annot=selected_rows.applymap(lambda x: f'{x}\n' if x != 0 else ''), fmt='', cmap='YlGnBu', cbar_kws={'label': 'Количество', 'aspect': 30}, annot_kws={"size": 8})
    plt.title(f'Errors Count Heatmap (Rows {start_row}-{start_row + num_rows})')
    plt.show()

In [ ]:
plot_heatmap(result_df, 0, 60, fig_all=True)

In [ ]:
plot_heatmap(result_df.sort_values(by=['Всего ошибок']), 0, 30, true_order=True)
plot_heatmap(result_df.sort_values(by=['Всего ошибок']), 30, 30, true_order=True)

In [ ]:
df3.sum(axis=1)

In [ ]:
df3[df3.sum(axis=1) == 0]

In [ ]:
2515 / 5980

# Загрузка нового датафрейма

In [ ]:
# Загрузка
df = pd.read_csv('/content/drive/MyDrive/df_info_new_ru.csv')

In [ ]:
df.columns

In [ ]:
df = df.sort_values(by='индекс файла')

In [ ]:
print(*df[df['нестандартная нижняя таблица'] == 1].filename_new[:10], sep='\n')

In [ ]:
def convert_to_range(x):
    lower_bound = (x // 100) * 100
    upper_bound = lower_bound + 100
    return f'{lower_bound}-{upper_bound}/'

# Примеры использования
x_values = [1, 100, 654, 1999]

for x in x_values:
    y = convert_to_range(x)
    print(f"Для x = {x}, y = {y}")